In [7]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import sys
import time
import copy
import string
import json
import pickle as pk
import math

from treelib import Node, Tree

from collections import Counter

from math import sin, asin, cos, radians, fabs, sqrt

In [13]:
############################    从文件中 读取 数据   ##############
def read_q(filename):
    f = open(filename,"rb+")
    st = pk.load(f)
    f.close()
    return st

In [14]:
##########################    向文件中  写 数据  #####################
def write_q(st, filename):
    f = open(filename, "wb+")
    pk.dump(st,f)
    f.close()

In [15]:
#######################   Dis()  根据经纬度计算距离 distance(a)  ####################3
#     input:  a  2*2 两个点的经纬度坐标；
#     output:  距离  m

################    测试用例   #################
#a = [[40.009532,116.322153],[40.009493,116.322103]]
#b = pd.DataFrame(a)
#Dis(b)

EARTH_RADIUS = 6371.004           #地球平均半径，6371km
 
def hav(theta):
    s = sin(theta / 2)
    return s * s
 
def Dis(b): 
#def get_distance_hav(lat0, lng0, lat1, lng1):
    """用haversine公式计算球面两点间的距离。"""
    # 经纬度转换成弧度
    a = pd.DataFrame(b)
    lat0 = radians(a.iloc[0,0])
    lat1 = radians(a.iloc[1,0])
    lng0 = radians(a.iloc[0,1])
    lng1 = radians(a.iloc[1,1])
 
    dlng = fabs(lng0 - lng1)
    dlat = fabs(lat0 - lat1)
    h = hav(dlat) + cos(lat0) * cos(lat1) * hav(dlng)
    dist = 2 * EARTH_RADIUS * asin(sqrt(h))*1000
 
    return dist

In [16]:
def tra_cos(a):
    b = a[0]
    c = a[1]
    d = a[2]
    
    bc = Dis([b,c])
    bd = Dis([b,d])
    cd = Dis([c,d])
    
    t1t2 = math.pow(bc, 2)
    t1r2 = math.pow(bd, 2)
    t2r2 = math.pow(cd, 2)
        
    cosva = (t1t2 + t1r2 - t2r2)/ (2*bc*bd)
    
    return cosva

In [17]:
###################################   单停留点与 POI的匹配    ############
# 输入:  停留点list (一个 三元组) , st_poi_dic 字典list  [{},{},{}]
# 输出： 最近的POI  及 停留时间  和 语义类型    五元组[id, lat,lng,time,category]
#st_p = st_p_list[0]

def match_1(st_p, sp_dic):
    poi_st_1 = [] 
    for i in range(0, len(sp_dic)):
        a = [[sp_dic[i]["location"]["lat"], sp_dic[i]["location"]["lng"]],[st_p[0],st_p[1]]]
        b = pd.DataFrame(a)
        c = Dis(b)
        poi_st_1.append(c)    
    #print(len(poi_st_1)) 
    j = poi_st_1.index( min(poi_st_1) )
    return  [sp_dic[j]["id"],sp_dic[j]["location"]["lat"], sp_dic[j]["location"]["lng"], st_p[2],sp_dic[j]["category"]]
    

In [18]:
###################  match()  停留点与语义树叶子结点的匹配  ##############################
# 输入 si_h_f[i]   一维list ; st_l   语义树的所有叶子结点
# 输出 sit_l    匹配后的叶子结点

#si = ['北京郊野园中园', 39.996209, 116.26688600000001, 13.75000013387762, 'farm', 1]
#st_l = st.leaves()
#n_id = match(si, st_l)
#st.get_node(n_id)

def match_2(si, st_l):
    for i in range(len(st_l)):
    
        if st_l[i].data == None:
            continue
        else:
        
            if si[0]==st_l[i].data["id"] and si[1]==st_l[i].data["location"]["lat"] and \
            si[2]==st_l[i].data["location"]["lng"]:
                n_id = st_l[i].identifier
                break

            
    return n_id       

In [19]:
from math import radians, cos, sin, asin, sqrt
def distance_vec(lon1, lat1, lon2_v, lat2_v): # 经度1，纬度1，经度2，纬度2 （十进制度数）
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)   """
    # 将十进制度数转化为弧度
    lon1, lat1 = map(radians, [lon1, lat1])
    lon2_v=np.radians(lon2_v)
    lat2_v=np.radians(lat2_v)
    dlon = lon2_v - lon1 
    dlat = lat2_v - lat1 
    a = np.power(np.sin(dlat/2),2) + cos(lat1) *np.cos(lat2_v) *  np.power(np.sin(dlon/2),2)#**2
    c = 2 *np.arcsin(np.sqrt(a)) 
    r = 6371 # 地球平均半径，单位为公里
    dis_l=c * r * 1000
    #print(dis_l.argmin())
    return dis_l

In [20]:
##########################    生成K-匿名集  ##################
## 输入 POI si;   候选匿名集 m_l;  k
## 输出 K-匿名集     k_as   [{},{},{},{}]

#si = ['北京郊野园中园', 39.996209, 116.26688600000001, 13.75000013387762, 'farm', 1]
#m_l = st.leaves("farm")
#k = 3
#a = gen_k_as(si, m_l, k)
#a

def gen_k_as(si, m_l, k):
    
    k_as = []
    #dis = []
    m_l2 = copy.deepcopy(m_l)
    
    #print("K-匿名集生成前叶子结点list长度： ",len(m_l2))

    ####  距离函数重写    #######
    lat = si[1]
    lng = si[2]
    
    m_l2_loc = []
    for i in range(len(m_l2)):
        n_d = m_l2[i].data
        a = []
        a = [n_d["location"]["lat"],n_d["location"]["lng"]]
        m_l2_loc.append(a)
        
    df_ml_list=pd.DataFrame(m_l2_loc,columns=['la','lo'])
    la_l=np.array(df_ml_list['la'])
    lo_l=np.array(df_ml_list['lo'])
    
    dis_l = distance_vec(lng, lat, lo_l, la_l)
    #print(dis_l)
    
    for l in range(len(m_l2)):    # 添加距离字段
        m_l2[l].data["distance"] = dis_l[l]
        
    ##########
            
    #print("计算距离后，打印第一个叶子结点： ", m_l2[0])
    m = sorted(m_l2,key = distance)      #自定义函数按列表f中字典的distance从小到大排序， 返回结点 list    
    del(m[0])
    
    if len(m) < k:
        k_as = m
    else:
        for j in range(k):
            k_as.append(m[j])
            
    return k_as   

def distance(s): 
    return s.data['distance'] 

In [21]:
####################   proba()  把 staying list  转换为概率特征    ############
#  输入: staying list   一个POI上的停留时间list ;  把停留时间化为了9个时段0::8
#  输出: st_f      停留时间概率分布，  
def proba(st):
    t = st.copy()
    for i in range(0, len(st)):
    
        if 10 <= st[i] < 30:
            t[i] = 0
        elif 30 <= st[i] < 60:
            t[i] = 1
        elif 60 <= st[i] < 90:
            t[i] = 2
        elif 90 <= st[i] < 120:
            t[i] = 3
        elif 120 <= st[i] < 150:
            t[i] = 4
        elif 150 <= st[i] < 180:
            t[i] = 5
        elif 180 <= st[i] < 210:
            t[i] = 6
        elif 210 <= st[i] < 240:
            t[i] = 7
        else:
            t[i] = 8
    
    b = len(t)
    a = Counter(t)
    #print(a)
    stf = [0]*9        # staying list 转化为 fecture vector
    #print(st_f)

    for key, value in a.items():
        stf[key] = round(value/b, 3)
        
    return stf   

In [22]:
def tra_cos(a):
    #b = a[0]
    #c = a[1]
    #d = a[2]
    
    bc = a[0]
    bd = a[1]
    cd = a[2]
    
    t1t2 = math.pow(bc, 2)
    t1r2 = math.pow(bd, 2)
    t2r2 = math.pow(cd, 2)
        
    cosva = (t1t2 + t1r2 - t2r2)/ (2*bc*bd)
    
    return cosva

In [23]:
##################################     读文件   换用户     ##########################################################
##########  读个人 st_poi_pro_p    语义重要性   ##########     用户： 000
i = 0 
filename = ".\\os_file\\public_data\\"+str(i)+"\\st_poi_pro_p"+str(i)+".pkl"
st_poi_pro_p = read_q(filename)
print(len(st_poi_pro_p))

5


In [13]:
###############  读文件，准备数据  ###########################
##########  总停留点  #####################
filename = ".\\os_file\\public_data\\st_pa.pkl"       #三重列表 [【user】【file】【st_p】]
st_pa = read_q(filename)
print(len(st_pa))

########## 总 st_poi_pro  语义重要性  ###############
filename = ".\\os_file\\public_data\\st_poi_pro.pkl"   #{'enter':{'bar':0.0041,'bath':0.03082}{}}{}{}{}{}
st_poi_pro = read_q(filename)
print(len(st_poi_pro))

##########  st_poi_dic  总ST-POI dict-list字典列表 ##################
filename = ".\\os_file\\public_data\\st_poi_dic.pkl"
st_poi_dic = read_q(filename)
print(len(st_poi_dic))

182
5
3634


1

In [ ]:
##############################   主程序   4*30（120） ----比较实验 ####################
## 输入 st_pa   所有停留点  三层list  [user][file][st_p]
## 输出 st_poi_f  五元组list   [id, lat, lng,time, category]

#### 从总停留点 st_pa 提取用户文件的停留点
##   用户： 000；  文件： 48， 80， 138， 147， 168 (j = 168-1)    
##   

big_t = [

[	"s_cluster"	,	"0_1"	,	"r_2"	,	3	,	2	],
[	"s_cluster"	,	"0_1"	,	"r_3"	,	3	,	3	],
[	"s_cluster"	,	"0_1"	,	"r_4"	,	3	,	4	],
[	"s_cluster"	,	"0_1"	,	"r_5"	,	3	,	5	],
[	"s_cluster"	,	"0_1"	,	"r_6"	,	3	,	6	],
[	"s_cluster"	,	"0_2"	,	"r_2"	,	3	,	2	],
[	"s_cluster"	,	"0_2"	,	"r_3"	,	3	,	3	],
[	"s_cluster"	,	"0_2"	,	"r_4"	,	3	,	4	],
[	"s_cluster"	,	"0_2"	,	"r_5"	,	3	,	5	],
[	"s_cluster"	,	"0_2"	,	"r_6"	,	3	,	6	],
[	"s_cluster"	,	"0_3"	,	"r_2"	,	3	,	2	],
[	"s_cluster"	,	"0_3"	,	"r_3"	,	3	,	3	],
[	"s_cluster"	,	"0_3"	,	"r_4"	,	3	,	4	],
[	"s_cluster"	,	"0_3"	,	"r_5"	,	3	,	5	],
[	"s_cluster"	,	"0_3"	,	"r_6"	,	3	,	6	]

]

"""kd = 2        # k 匿名度
o_r = 3     #  最高高度
o_l = 1     #  最低高度
cluster = "mb_kmeans"    # 聚类器
ex_times = "0_1"      #第几次实验
at = "k_2"      # 参数设置
i = 0
"""

i = 0
o_l = 1     #  最低高度
num = 0
num_tot = 0

for bb in big_t:
    cluster = bb[0]   # 聚类器  a_cluster,,mb_kmeans
    ex_times = bb[1]      #第几次实验
    at = bb[2]
    kd = bb[3]        # k 匿名度
    o_r = bb[4]    #  最高高度
    print(cluster, ex_times, at,kd,o_r)
    
    ########## semantic_tree    ###############
    filename = ".\\os_file\\st_poi_clusters\\"+str(cluster)+"\\semantic_tree.pkl"
    st = read_q(filename)

    wz = [47,79,137,146,167]
    for aa in wz:
        j = aa
    #j = 167   # 第48个文件  48   138   168

        filename1 = ".\\os_file\\res_8\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\st_poi_f_"+str(i)+"_"+str(j)+".pkl"
        filename2 = ".\\os_file\\res_8\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\si_l_f_"+str(i)+"_"+str(j)+".pkl"
        filename3 = ".\\os_file\\res_8\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\si_h_f_"+str(i)+"_"+str(j)+".pkl"
        filename4 = ".\\os_file\\res_8\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\k_as_"+str(i)+"_"+str(j)+".pkl" 
        filename5 = ".\\os_file\\res_8\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\rec_tra_"+str(i)+"_"+str(j)+".pkl"
        filename6 = ".\\os_file\\res_8\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\rec_cos_"+str(i)+"_"+str(j)+".pkl"
        filename7 = ".\\os_file\\res_8\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\tra_leaf_"+str(i)+"_"+str(j)+".pkl"

#############################################################
#  输入  st_pa 所有停留点   三重列表
#  输出  st_pa_f  文件的停留点    一维列表  取文件中的停留点

        st_pa_f = []
#st_p = []    #  停留点的list 

        m = st_pa

        if len(m[i]) == 0:            #  如果  user i 没有文件 
            print("该用户没有轨迹文件")
            sys.exit()
        else:       
            if len(m[i][j]) == 0:       #  文件内没有stay point，
                print("该文件内没有停留点")
                sys.exit()
            else:      #  文件内有 stay point               #  user_file_st-p
                for k in range(0, len(m[i][j])):
                            # 对用户 i 文件 j, 停留点 k, st_p[i][j][k]     停留点
                    a = []
                    a = m[i][j][k]
                    st_pa_f.append(a)
            
        b = 0
        b = len(st_pa_f)
        num_tot += b
        print("用户： ",i," 在文件： ",j," 中共有停留点有： ", b,"  个")

#################  匹配 st_pa_f ----> st_poi_f ############### 
############   把 文件停留点， 经 st_poi_dic 匹配, 得 POI+ category： st_poi_f
# 输入  文件停留点st_pa_f,    st_poi_dic   停留点与POI匹配的字典列表
# 输出  st_poi_f      文件与POI匹配列表   二维列表
# [['无尽', 40.005973, 116.32533400000001, 65.81666663987562, 'scenic'],

        st_poi_f = []
        for l in range(0, len(st_pa_f)):
#for i in range(0, 20):
    
            st_p_m = match_1(st_pa_f[l], st_poi_dic)
            st_poi_f.append(st_p_m)

        write_q(st_poi_f, filename1)
        print("已写入 st_poi_f 文件")

##############################  semantic importance  ##################
## 输入 st_poi_f  五元组-两维list   [[id, lat, lng,time, category],  ,  ] ;    st_poi_pro_p, st_poi_pro
## 输出 si_l_f    六元组-两维list   [id, lat, lng,time, category,level]
# 输出文件 si_l_f_0_47.pkl   

        f = st_poi_f
        pro_p = st_poi_pro_p
        pro = st_poi_pro

        si_l_f = []
        for k in range(len(f)):
    
            ca = f[k][4]
    
            flag = 0
    
            for item1 in pro:
        
                if flag == 1:
                    break
                else:
            
                    for item2 in pro[item1]:
                        if flag == 1:
                            break
                        else:
                            if ca == item2:         # 判断语义类型匹配
                                flag = 1
                                a = 0
                                a = pro[item1][item2]
                                b = 0
                                b = pro_p[item1][item2]
                        
                                if a == 0:
                                    print("st_poi_pro 没有该类型的停留点")
                            
                                c=0
                                c = b * math.log(1/a)
                        
                                d = [f[k][0],f[k][1],f[k][2],f[k][3],f[k][4],c]
                                si_l_f.append(d)
                    
        write_q(si_l_f, filename2)
        print("已写入 si_l_f 文件")
###########################  si_h_f 语义树高度   #######################
## 输入 si_l_f， o_l, o_r   
## 输出 si_h_f     六元组-两维list   [[id, lat, lng,time, category,height], , ]
                  # 输出文件 si_h_f_0_47.pkl 

        al = []
        for k in range(len(si_l_f)):
            al.append(si_l_f[k][5])
        w = max(al)

        si_h_f = []
        for k in range(len(si_l_f)):
            a = si_l_f[k][5]
            b = a/w
            c = 1-b
    
            l = b * o_l + c *o_r
            h = round(l)
    
            d = []
            d = [si_l_f[k][0],si_l_f[k][1],si_l_f[k][2],si_l_f[k][3],si_l_f[k][4],h]
            si_h_f.append(d)

        write_q(si_h_f, filename3)
        print("已写入 si_h_f 文件")
#####################  轨迹重构 rec_tra  rec_tra_i_j.pkl ################
# 输入  si_h_f 文件内停留点的语义高度； st 语义树；
# 重构轨迹 rec_tra  

        si = si_h_f    # 文件中停留点list 6元组
        st_l = st.leaves()   # st 叶子结点list

        rec_tra = []   # 存储 重构的轨迹
        k_as = []      # 存储 K 匿名集
        rec_cos = []    #  角度
        tra_leaf = []   # 存储匹配的原始叶子结点
        for k in range(len(si)):
    
            h = si[k][5]      # 语义匿名高度
            n_id = match_2(si[k], st_l)     # 文件停留点与叶子结点匹配，   返回匹配结点  id
            sp_node = st.get_node(n_id)
            tra_leaf.append(sp_node)
    #print(st.level(n_id)+1, h)
            m_id = n_id
            if st.level(m_id) < h:
                m_id = "poi"
            else:
                for l in range(h):
            #print(l)
                    m_id = st.parent(m_id).identifier   # 返回 中间结点 id
    
    #print(m_id)
            m_l = st.leaves(m_id)    # 返回 中间结点 对应的 叶子结点  匿名集  [node, node, node]
    ##############  删除 空结点  #############
            ai = 0
            while ai < len(m_l):     #  删除空叶子结点
        
                if m_l[ai].data == None:
                    del m_l[ai]
                else:
                    ai += 1
   
    ########################################
            for l in range(len(m_l)):         ## staying 特征概率化
                abc = m_l[l].data["staying"]
                bcd = proba(abc)
                m_l[l].data["staying"] = bcd
    ########################### 选点  ##############################   
    #  对轨迹第一个点，生成点只要在1500范围内即可
    #  对轨迹中连续的相同点，条件同上
    #  对轨迹中非同一点，角要小于 53 度
            k_as_m = gen_k_as(si[k], m_l, kd)       # 生成K-匿名集，  [node,node,node]
            print("匿名集的大小", len(k_as_m))
            #print("已生成 si_h_f:")
            t2 = [si[k][1],si[k][2]]  
            print("文件：",j,"当前是第",k,"个点")
            
            if len(k_as_m) == 0:         #  如果匿名集为空，返回原结点
                recon_node = sp_node
                num += 1
            else:
                
                if k == 0:               # 对轨迹第一个结点，要求在 1.5km 内生成 fake 结点
                    flag = 0
                    for aa in k_as_m:               
                        r2 = [aa.data["location"]["lat"],aa.data["location"]["lng"]]
                        dis = Dis([t2, r2])
                        aa.data["dis"] = dis
                        if dis < 1500:
                            flag = 1
                            
                    if flag == 0:                              # 判断是否存在 1.5km 内的点
                        recon_node = sp_node
                        num += 1
                    else:
                        flag = 0
                        while flag == 0:
                            idx = np.random.randint(len(k_as_m))      # k范围内，随机生成索引
                            dis = k_as_m[idx].data["dis"]
                            print("轨迹第一个停留点， 生成点与原始点距离为：", dis)
                            if dis < 1500:
                                flag = 1
                        recon_node = k_as_m[idx]
                else:
                    t1 = [si[k-1][1],si[k-1][2]]
                    t1_t2 = Dis([t1,t2])
            
                    if t1_t2 == 0:         #  如果是同一个停留点，则只要距离不超过  1500 米就行
                        print("该轨迹点与前一个点相同")
                        flag = 0
                        for aa in k_as_m:               
                            r2 = [aa.data["location"]["lat"],aa.data["location"]["lng"]]
                            dis = Dis([t2, r2])
                            aa.data["dis"] = dis
                            if dis < 1500:
                                flag = 1
                            
                        if flag == 0:                              # 判断是否存在 1.5km 内的点
                            recon_node = sp_node
                            num += 1
                        else:
                            flag = 0
                            while flag == 0:
                                idx = np.random.randint(len(k_as_m))      # k范围内，随机生成索引
                                dis = k_as_m[idx].data["dis"]
                                print("生成点与原始点距离为：", dis)
                                if dis < 1500:
                                    flag = 1
                            recon_node = k_as_m[idx]
                        """
                        flag = 0
                        while flag == 0:
                            idx = np.random.randint(len(k_as_m))      # k范围内，随机生成索引
                            r2 = [k_as_m[idx].data["location"]["lat"],k_as_m[idx].data["location"]["lng"]]
                    
                            t2_r2 = Dis([t2,r2])
                            print("该轨迹点与生成点距离为： ", t2_r2)
                            if t2_r2 < 1500:
                                flag = 1
                        recon_node = k_as_m[idx]  """
                    else:
                
                        print("该轨迹点既不是第一个，与前一个点也不相同")
                        flag = 0
                        for aa in k_as_m:
                            r2 = [aa.data["location"]["lat"],aa.data["location"]["lng"]]
                            t1_r2 = Dis([t1,r2])
                            t2_r2 = Dis([t2,r2])
                            if t1_r2 == 0:     #       如果 fake 位置与前一个位置相同，则放弃
                                aa.data["cos"] = -1
                            else:                   #   正常结点， 在90度偏移范围内选择 fake 结点 
                                dista = [t1_t2, t1_r2, t2_r2]
                                ttcos = tra_cos(dista)
                                aa.data["cos"] = ttcos
                                if ttcos >= 0:
                                    flag = 1
                    
                        if flag == 1:
                            tcos = -1
                            while tcos < 0:
                                idx = np.random.randint(len(k_as_m))      # k范围内，随机生成索引
                                tcos = k_as_m[idx].data["cos"]
                        #print("当前生成点与原始轨迹两个点的偏角余弦为：",tcos)
                            recon_node = k_as_m[idx]
                        else:
                            tcos = 1
                            sp_node.data["cos"] = tcos
                            recon_node = sp_node
                            num += 1 
                        
                        rec_cos.append(tcos)

            rec_tra.append(recon_node)
            k_as.append(k_as_m)
        
        write_q(tra_leaf, filename7)
        write_q(rec_tra, filename5)
        write_q(k_as,filename4)
        write_q(rec_cos,filename6)
        print("用户在文件: ",j,"中共有重构位置：",len(rec_tra),"个")
print("在为用户重构轨迹过程中共保留原始结点：",num,"个；重构结点总数为：",num_tot,"个；比例为：",round(num/num_tot,4))

In [14]:
##############################   主程序   4*30（120）  ####################
## 输入 st_pa   所有停留点  三层list  [user][file][st_p]
## 输出 st_poi_f  五元组list   [id, lat, lng,time, category]

#### 从总停留点 st_pa 提取用户文件的停留点
##   用户： 000；  文件： 48， 80， 138， 147， 168 (j = 168-1)    
##   

big_t = [

[	"s_cluster"	,	"0_1"	,	"k_2"	,	2	,	3	],
[	"s_cluster"	,	"0_1"	,	"k_3"	,	3	,	3	],
[	"s_cluster"	,	"0_1"	,	"k_4"	,	4	,	3	],
[	"s_cluster"	,	"0_1"	,	"k_5"	,	5	,	3	],
[	"s_cluster"	,	"0_1"	,	"k_6"	,	6	,	3	],
[	"s_cluster"	,	"0_1"	,	"r_2"	,	3	,	2	],
[	"s_cluster"	,	"0_1"	,	"r_3"	,	3	,	3	],
[	"s_cluster"	,	"0_1"	,	"r_4"	,	3	,	4	],
[	"s_cluster"	,	"0_1"	,	"r_5"	,	3	,	5	],
[	"s_cluster"	,	"0_1"	,	"r_6"	,	3	,	6	],
[	"s_cluster"	,	"0_2"	,	"k_2"	,	2	,	3	],
[	"s_cluster"	,	"0_2"	,	"k_3"	,	3	,	3	],
[	"s_cluster"	,	"0_2"	,	"k_4"	,	4	,	3	],
[	"s_cluster"	,	"0_2"	,	"k_5"	,	5	,	3	],
[	"s_cluster"	,	"0_2"	,	"k_6"	,	6	,	3	],
[	"s_cluster"	,	"0_2"	,	"r_2"	,	3	,	2	],
[	"s_cluster"	,	"0_2"	,	"r_3"	,	3	,	3	],
[	"s_cluster"	,	"0_2"	,	"r_4"	,	3	,	4	],
[	"s_cluster"	,	"0_2"	,	"r_5"	,	3	,	5	],
[	"s_cluster"	,	"0_2"	,	"r_6"	,	3	,	6	],
[	"s_cluster"	,	"0_3"	,	"k_2"	,	2	,	3	],
[	"s_cluster"	,	"0_3"	,	"k_3"	,	3	,	3	],
[	"s_cluster"	,	"0_3"	,	"k_4"	,	4	,	3	],
[	"s_cluster"	,	"0_3"	,	"k_5"	,	5	,	3	],
[	"s_cluster"	,	"0_3"	,	"k_6"	,	6	,	3	],
[	"s_cluster"	,	"0_3"	,	"r_2"	,	3	,	2	],
[	"s_cluster"	,	"0_3"	,	"r_3"	,	3	,	3	],
[	"s_cluster"	,	"0_3"	,	"r_4"	,	3	,	4	],
[	"s_cluster"	,	"0_3"	,	"r_5"	,	3	,	5	],
[	"s_cluster"	,	"0_3"	,	"r_6"	,	3	,	6	],
[	"k_means"	,	"0_1"	,	"k_2"	,	2	,	3	],
[	"k_means"	,	"0_1"	,	"k_3"	,	3	,	3	],
[	"k_means"	,	"0_1"	,	"k_4"	,	4	,	3	],
[	"k_means"	,	"0_1"	,	"k_5"	,	5	,	3	],
[	"k_means"	,	"0_1"	,	"k_6"	,	6	,	3	],
[	"k_means"	,	"0_1"	,	"r_2"	,	3	,	2	],
[	"k_means"	,	"0_1"	,	"r_3"	,	3	,	3	],
[	"k_means"	,	"0_1"	,	"r_4"	,	3	,	4	],
[	"k_means"	,	"0_1"	,	"r_5"	,	3	,	5	],
[	"k_means"	,	"0_1"	,	"r_6"	,	3	,	6	],
[	"k_means"	,	"0_2"	,	"k_2"	,	2	,	3	],
[	"k_means"	,	"0_2"	,	"k_3"	,	3	,	3	],
[	"k_means"	,	"0_2"	,	"k_4"	,	4	,	3	],
[	"k_means"	,	"0_2"	,	"k_5"	,	5	,	3	],
[	"k_means"	,	"0_2"	,	"k_6"	,	6	,	3	],
[	"k_means"	,	"0_2"	,	"r_2"	,	3	,	2	],
[	"k_means"	,	"0_2"	,	"r_3"	,	3	,	3	],
[	"k_means"	,	"0_2"	,	"r_4"	,	3	,	4	],
[	"k_means"	,	"0_2"	,	"r_5"	,	3	,	5	],
[	"k_means"	,	"0_2"	,	"r_6"	,	3	,	6	],
[	"k_means"	,	"0_3"	,	"k_2"	,	2	,	3	],
[	"k_means"	,	"0_3"	,	"k_3"	,	3	,	3	],
[	"k_means"	,	"0_3"	,	"k_4"	,	4	,	3	],
[	"k_means"	,	"0_3"	,	"k_5"	,	5	,	3	],
[	"k_means"	,	"0_3"	,	"k_6"	,	6	,	3	],
[	"k_means"	,	"0_3"	,	"r_2"	,	3	,	2	],
[	"k_means"	,	"0_3"	,	"r_3"	,	3	,	3	],
[	"k_means"	,	"0_3"	,	"r_4"	,	3	,	4	],
[	"k_means"	,	"0_3"	,	"r_5"	,	3	,	5	],
[	"k_means"	,	"0_3"	,	"r_6"	,	3	,	6	],
[	"a_cluster"	,	"0_1"	,	"k_2"	,	2	,	3	],
[	"a_cluster"	,	"0_1"	,	"k_3"	,	3	,	3	],
[	"a_cluster"	,	"0_1"	,	"k_4"	,	4	,	3	],
[	"a_cluster"	,	"0_1"	,	"k_5"	,	5	,	3	],
[	"a_cluster"	,	"0_1"	,	"k_6"	,	6	,	3	],
[	"a_cluster"	,	"0_1"	,	"r_2"	,	3	,	2	],
[	"a_cluster"	,	"0_1"	,	"r_3"	,	3	,	3	],
[	"a_cluster"	,	"0_1"	,	"r_4"	,	3	,	4	],
[	"a_cluster"	,	"0_1"	,	"r_5"	,	3	,	5	],
[	"a_cluster"	,	"0_1"	,	"r_6"	,	3	,	6	],
[	"a_cluster"	,	"0_2"	,	"k_2"	,	2	,	3	],
[	"a_cluster"	,	"0_2"	,	"k_3"	,	3	,	3	],
[	"a_cluster"	,	"0_2"	,	"k_4"	,	4	,	3	],
[	"a_cluster"	,	"0_2"	,	"k_5"	,	5	,	3	],
[	"a_cluster"	,	"0_2"	,	"k_6"	,	6	,	3	],
[	"a_cluster"	,	"0_2"	,	"r_2"	,	3	,	2	],
[	"a_cluster"	,	"0_2"	,	"r_3"	,	3	,	3	],
[	"a_cluster"	,	"0_2"	,	"r_4"	,	3	,	4	],
[	"a_cluster"	,	"0_2"	,	"r_5"	,	3	,	5	],
[	"a_cluster"	,	"0_2"	,	"r_6"	,	3	,	6	],
[	"a_cluster"	,	"0_3"	,	"k_2"	,	2	,	3	],
[	"a_cluster"	,	"0_3"	,	"k_3"	,	3	,	3	],
[	"a_cluster"	,	"0_3"	,	"k_4"	,	4	,	3	],
[	"a_cluster"	,	"0_3"	,	"k_5"	,	5	,	3	],
[	"a_cluster"	,	"0_3"	,	"k_6"	,	6	,	3	],
[	"a_cluster"	,	"0_3"	,	"r_2"	,	3	,	2	],
[	"a_cluster"	,	"0_3"	,	"r_3"	,	3	,	3	],
[	"a_cluster"	,	"0_3"	,	"r_4"	,	3	,	4	],
[	"a_cluster"	,	"0_3"	,	"r_5"	,	3	,	5	],
[	"a_cluster"	,	"0_3"	,	"r_6"	,	3	,	6	],
[	"mb_kmeans"	,	"0_1"	,	"k_2"	,	2	,	3	],
[	"mb_kmeans"	,	"0_1"	,	"k_3"	,	3	,	3	],
[	"mb_kmeans"	,	"0_1"	,	"k_4"	,	4	,	3	],
[	"mb_kmeans"	,	"0_1"	,	"k_5"	,	5	,	3	],
[	"mb_kmeans"	,	"0_1"	,	"k_6"	,	6	,	3	],
[	"mb_kmeans"	,	"0_1"	,	"r_2"	,	3	,	2	],
[	"mb_kmeans"	,	"0_1"	,	"r_3"	,	3	,	3	],
[	"mb_kmeans"	,	"0_1"	,	"r_4"	,	3	,	4	],
[	"mb_kmeans"	,	"0_1"	,	"r_5"	,	3	,	5	],
[	"mb_kmeans"	,	"0_1"	,	"r_6"	,	3	,	6	],
[	"mb_kmeans"	,	"0_2"	,	"k_2"	,	2	,	3	],
[	"mb_kmeans"	,	"0_2"	,	"k_3"	,	3	,	3	],
[	"mb_kmeans"	,	"0_2"	,	"k_4"	,	4	,	3	],
[	"mb_kmeans"	,	"0_2"	,	"k_5"	,	5	,	3	],
[	"mb_kmeans"	,	"0_2"	,	"k_6"	,	6	,	3	],
[	"mb_kmeans"	,	"0_2"	,	"r_2"	,	3	,	2	],
[	"mb_kmeans"	,	"0_2"	,	"r_3"	,	3	,	3	],
[	"mb_kmeans"	,	"0_2"	,	"r_4"	,	3	,	4	],
[	"mb_kmeans"	,	"0_2"	,	"r_5"	,	3	,	5	],
[	"mb_kmeans"	,	"0_2"	,	"r_6"	,	3	,	6	],
[	"mb_kmeans"	,	"0_3"	,	"k_2"	,	2	,	3	],
[	"mb_kmeans"	,	"0_3"	,	"k_3"	,	3	,	3	],
[	"mb_kmeans"	,	"0_3"	,	"k_4"	,	4	,	3	],
[	"mb_kmeans"	,	"0_3"	,	"k_5"	,	5	,	3	],
[	"mb_kmeans"	,	"0_3"	,	"k_6"	,	6	,	3	],
[	"mb_kmeans"	,	"0_3"	,	"r_2"	,	3	,	2	],
[	"mb_kmeans"	,	"0_3"	,	"r_3"	,	3	,	3	],
[	"mb_kmeans"	,	"0_3"	,	"r_4"	,	3	,	4	],
[	"mb_kmeans"	,	"0_3"	,	"r_5"	,	3	,	5	],
[	"mb_kmeans"	,	"0_3"	,	"r_6"	,	3	,	6	]

  
]

"""kd = 2        # k 匿名度
o_r = 3     #  最高高度
o_l = 1     #  最低高度
cluster = "mb_kmeans"    # 聚类器
ex_times = "0_1"      #第几次实验
at = "k_2"      # 参数设置
i = 0
"""

i = 0
o_l = 1     #  最低高度
num = 0
num_tot = 0

for bb in big_t:
    cluster = bb[0]   # 聚类器  a_cluster,,mb_kmeans
    ex_times = bb[1]      #第几次实验
    at = bb[2]
    kd = bb[3]        # k 匿名度
    o_r = bb[4]    #  最高高度
    print(cluster, ex_times, at,kd,o_r)
    
    ########## semantic_tree    ###############
    filename = ".\\os_file\\st_poi_clusters\\"+str(cluster)+"\\semantic_tree.pkl"
    st = read_q(filename)

    wz = [47,79,137,146,167]
    for aa in wz:
        j = aa
    #j = 167   # 第48个文件  48   138   168

        filename1 = ".\\os_file\\res_6\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\st_poi_f_"+str(i)+"_"+str(j)+".pkl"
        filename2 = ".\\os_file\\res_6\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\si_l_f_"+str(i)+"_"+str(j)+".pkl"
        filename3 = ".\\os_file\\res_6\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\si_h_f_"+str(i)+"_"+str(j)+".pkl"
        filename4 = ".\\os_file\\res_6\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\k_as_"+str(i)+"_"+str(j)+".pkl" 
        filename5 = ".\\os_file\\res_6\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\rec_tra_"+str(i)+"_"+str(j)+".pkl"
        filename6 = ".\\os_file\\res_6\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\rec_cos_"+str(i)+"_"+str(j)+".pkl"
        filename7 = ".\\os_file\\res_6\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\tra_leaf_"+str(i)+"_"+str(j)+".pkl"

#############################################################
#  输入  st_pa 所有停留点   三重列表
#  输出  st_pa_f  文件的停留点    一维列表

        st_pa_f = []
#st_p = []    #  停留点的list 

        m = st_pa

        if len(m[i]) == 0:            #  如果  user i 没有文件 
            print("该用户没有轨迹文件")
            sys.exit()
        else:       
            if len(m[i][j]) == 0:       #  文件内没有stay point，
                print("该文件内没有停留点")
                sys.exit()
            else:      #  文件内有 stay point               #  user_file_st-p
                for k in range(0, len(m[i][j])):
                            # 对用户 i 文件 j, 停留点 k, st_p[i][j][k]     停留点
                    a = []
                    a = m[i][j][k]
                    st_pa_f.append(a)
            
        b = 0
        b = len(st_pa_f)
        num_tot += b
        print("用户： ",i," 在文件： ",j," 中共有停留点有： ", b,"  个")

#################  匹配 st_pa_f ----> st_poi_f ############### 
############   把 文件停留点， 经 st_poi_dic 匹配, 得 POI+ category： st_poi_f
# 输入  文件停留点st_pa_f,    st_poi_dic   停留点与POI匹配的字典列表
# 输出  st_poi_f      文件与POI匹配列表   二维列表
# [['无尽', 40.005973, 116.32533400000001, 65.81666663987562, 'scenic'],

        st_poi_f = []
        for l in range(0, len(st_pa_f)):
#for i in range(0, 20):
    
            st_p_m = match_1(st_pa_f[l], st_poi_dic)
            st_poi_f.append(st_p_m)

        write_q(st_poi_f, filename1)
        print("已写入 st_poi_f 文件")

##############################  semantic importance  ##################
## 输入 st_poi_f  五元组-两维list   [id, lat, lng,time, category] ;    st_poi_pro_p, st_poi_pro
## 输出 si_l_f    六元组-两维list   [id, lat, lng,time, category,level]
# 输出文件 si_l_f_0_47.pkl   

        f = st_poi_f
        pro_p = st_poi_pro_p
        pro = st_poi_pro

        si_l_f = []
        for k in range(len(f)):
    
            ca = f[k][4]
    
            flag = 0
    
            for item1 in pro:
        
                if flag == 1:
                    break
                else:
            
                    for item2 in pro[item1]:
                        if flag == 1:
                            break
                        else:
                            if ca == item2:         # 判断语义类型匹配
                                flag = 1
                                a = 0
                                a = pro[item1][item2]
                                b = 0
                                b = pro_p[item1][item2]
                        
                                if a == 0:
                                    print("st_poi_pro 没有该类型的停留点")
                            
                                c=0
                                c = b * math.log(1/a)
                        
                                d = [f[k][0],f[k][1],f[k][2],f[k][3],f[k][4],c]
                                si_l_f.append(d)
                    
        write_q(si_l_f, filename2)
        print("已写入 si_l_f 文件")
###########################  si_h_f 语义树高度   #######################
## 输入 si_l_f， o_l, o_r   
## 输出 si_h_f     六元组-两维list   [id, lat, lng,time, category,height]
                  # 输出文件 si_h_f_0_47.pkl 

        al = []
        for k in range(len(si_l_f)):
            al.append(si_l_f[k][5])
        w = max(al)

        si_h_f = []
        for k in range(len(si_l_f)):
            a = si_l_f[k][5]
            b = a/w
            c = 1-b
    
            l = b * o_l + c *o_r
            h = round(l)
    
            d = []
            d = [si_l_f[k][0],si_l_f[k][1],si_l_f[k][2],si_l_f[k][3],si_l_f[k][4],h]
            si_h_f.append(d)

        write_q(si_h_f, filename3)
        print("已写入 si_h_f 文件")
#####################  轨迹重构 rec_tra  rec_tra_i_j.pkl ################
# 输入  si_h_f 文件内停留点的语义高度； st 语义树；
# 重构轨迹 rec_tra  

        si = si_h_f    # 文件中停留点list 6元组
        st_l = st.leaves()   # st 叶子结点list

        rec_tra = []   # 存储 重构的轨迹
        k_as = []      # 存储 K 匿名集
        rec_cos = []    #  角度
        tra_leaf = []   # 存储匹配的原始叶子结点
        for k in range(len(si)):
    
            h = si[k][5]      # 语义匿名高度
            n_id = match_2(si[k], st_l)     # 文件停留点与叶子结点匹配，   返回匹配结点  id
            sp_node = st.get_node(n_id)
            tra_leaf.append(sp_node)
    #print(st.level(n_id)+1, h)
            m_id = n_id
            if st.level(m_id) < h:
                m_id = "poi"
            else:
                for l in range(h):
            #print(l)
                    m_id = st.parent(m_id).identifier   # 返回 中间结点 id
    
    #print(m_id)
            m_l = st.leaves(m_id)    # 返回 中间结点 对应的 叶子结点  匿名集  [node, node, node]
    ##############  删除 空结点  #############
            ai = 0
            while ai < len(m_l):     #  删除空叶子结点
        
                if m_l[ai].data == None:
                    del m_l[ai]
                else:
                    ai += 1
   
    ########################################
            for l in range(len(m_l)):         ## staying 特征概率化
                abc = m_l[l].data["staying"]
                bcd = proba(abc)
                m_l[l].data["staying"] = bcd
    ########################### 选点  ##############################   
    #  对轨迹第一个点，生成点只要在1500范围内即可
    #  对轨迹中连续的相同点，条件同上
    #  对轨迹中非同一点，角要小于 53 度
            k_as_m = gen_k_as(si[k], m_l, kd)       # 生成K-匿名集，  [node,node,node]
            print("匿名集的大小", len(k_as_m))
            #print("已生成 si_h_f:")
            t2 = [si[k][1],si[k][2]]  
            print("文件：",j,"当前是第",k,"个点")
            
            if len(k_as_m) == 0:         #  如果匿名集为空，返回原结点
                recon_node = sp_node
                num += 1
            else:
                
                if k == 0:               # 对轨迹第一个结点，要求在 1.5km 内生成 fake 结点
                    flag = 0
                    for aa in k_as_m:               
                        r2 = [aa.data["location"]["lat"],aa.data["location"]["lng"]]
                        dis = Dis([t2, r2])
                        aa.data["dis"] = dis
                        if dis < 1500:
                            flag = 1
                            
                    if flag == 0:                              # 判断是否存在 1.5km 内的点
                        recon_node = sp_node
                        num += 1
                    else:
                        flag = 0
                        while flag == 0:
                            idx = np.random.randint(len(k_as_m))      # k范围内，随机生成索引
                            dis = k_as_m[idx].data["dis"]
                            print("轨迹第一个停留点， 生成点与原始点距离为：", dis)
                            if dis < 1500:
                                flag = 1
                        recon_node = k_as_m[idx]
                else:
                    t1 = [si[k-1][1],si[k-1][2]]
                    t1_t2 = Dis([t1,t2])
            
                    if t1_t2 == 0:         #  如果是同一个停留点，则只要距离不超过  1500 米就行
                        print("该轨迹点与前一个点相同")
                        flag = 0
                        for aa in k_as_m:               
                            r2 = [aa.data["location"]["lat"],aa.data["location"]["lng"]]
                            dis = Dis([t2, r2])
                            aa.data["dis"] = dis
                            if dis < 1500:
                                flag = 1
                            
                        if flag == 0:                              # 判断是否存在 1.5km 内的点
                            recon_node = sp_node
                            num += 1
                        else:
                            flag = 0
                            while flag == 0:
                                idx = np.random.randint(len(k_as_m))      # k范围内，随机生成索引
                                dis = k_as_m[idx].data["dis"]
                                print("生成点与原始点距离为：", dis)
                                if dis < 1500:
                                    flag = 1
                            recon_node = k_as_m[idx]
                        """
                        flag = 0
                        while flag == 0:
                            idx = np.random.randint(len(k_as_m))      # k范围内，随机生成索引
                            r2 = [k_as_m[idx].data["location"]["lat"],k_as_m[idx].data["location"]["lng"]]
                    
                            t2_r2 = Dis([t2,r2])
                            print("该轨迹点与生成点距离为： ", t2_r2)
                            if t2_r2 < 1500:
                                flag = 1
                        recon_node = k_as_m[idx]  """
                    else:
                
                        print("该轨迹点既不是第一个，与前一个点也不相同")
                        flag = 0
                        for aa in k_as_m:
                            r2 = [aa.data["location"]["lat"],aa.data["location"]["lng"]]
                            t1_r2 = Dis([t1,r2])
                            t2_r2 = Dis([t2,r2])
                            if t1_r2 == 0:     #       如果 fake 位置与前一个位置相同，则放弃
                                aa.data["cos"] = -1
                            else:                   #   正常结点， 在90度偏移范围内选择 fake 结点 
                                dista = [t1_t2, t1_r2, t2_r2]
                                ttcos = tra_cos(dista)
                                aa.data["cos"] = ttcos
                                if ttcos >= 0:
                                    flag = 1
                    
                        if flag == 1:
                            tcos = -1
                            while tcos < 0:
                                idx = np.random.randint(len(k_as_m))      # k范围内，随机生成索引
                                tcos = k_as_m[idx].data["cos"]
                        #print("当前生成点与原始轨迹两个点的偏角余弦为：",tcos)
                            recon_node = k_as_m[idx]
                        else:
                            tcos = 1
                            sp_node.data["cos"] = tcos
                            recon_node = sp_node
                            num += 1 
                        
                        rec_cos.append(tcos)

            rec_tra.append(recon_node)
            k_as.append(k_as_m)
        
        write_q(tra_leaf, filename7)
        write_q(rec_tra, filename5)
        write_q(k_as,filename4)
        write_q(rec_cos,filename6)
        print("用户在文件: ",j,"中共有重构位置：",len(rec_tra),"个")
print("在为用户重构轨迹过程中共保留原始结点：",num,"个；重构结点总数为：",num_tot,"个；比例为：",round(num/num_tot,4))

s_cluster 0_1 k_2 2 3
用户：  0  在文件：  47  中共有停留点有：  14   个
已写入 st_poi_f 文件
已写入 si_l_f 文件
已写入 si_h_f 文件
匿名集的大小 2
文件： 47 当前是第 0 个点
轨迹第一个停留点， 生成点与原始点距离为： 115.78738887892789
匿名集的大小 2
文件： 47 当前是第 1 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 2
文件： 47 当前是第 2 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 2
文件： 47 当前是第 3 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 2
文件： 47 当前是第 4 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 2
文件： 47 当前是第 5 个点
该轨迹点与前一个点相同
生成点与原始点距离为： 202.5715932135442
匿名集的大小 2
文件： 47 当前是第 6 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 2
文件： 47 当前是第 7 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 2
文件： 47 当前是第 8 个点
该轨迹点与前一个点相同
生成点与原始点距离为： 132.98175918439696
匿名集的大小 2
文件： 47 当前是第 9 个点
该轨迹点与前一个点相同
生成点与原始点距离为： 141.65090375789288
匿名集的大小 2
文件： 47 当前是第 10 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 2
文件： 47 当前是第 11 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 2
文件： 47 当前是第 12 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 2
文件： 47 当前是第 13 个点
该轨迹点既不是第一个，与前一个点也不相同
用户在文件:  47 中共有重构位置： 14 个
用户：  0  在文件：  79  中共有停留点有：  10   个
已写入 st_poi_f 文件
已写入 si_l_f 文件
已写入 si_h_f 文件
匿名集的大小 2
文件： 79 当前是第 0 个点
轨迹第一个停留点， 生成点与原始点距离为： 

已写入 st_poi_f 文件
已写入 si_l_f 文件
已写入 si_h_f 文件
匿名集的大小 4
文件： 137 当前是第 0 个点
轨迹第一个停留点， 生成点与原始点距离为： 405.0295528000973
匿名集的大小 4
文件： 137 当前是第 1 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 4
文件： 137 当前是第 2 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 4
文件： 137 当前是第 3 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 4
文件： 137 当前是第 4 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 4
文件： 137 当前是第 5 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 4
文件： 137 当前是第 6 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 4
文件： 137 当前是第 7 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 4
文件： 137 当前是第 8 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 4
文件： 137 当前是第 9 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 4
文件： 137 当前是第 10 个点
该轨迹点既不是第一个，与前一个点也不相同
用户在文件:  137 中共有重构位置： 11 个
用户：  0  在文件：  146  中共有停留点有：  9   个
已写入 st_poi_f 文件
已写入 si_l_f 文件
已写入 si_h_f 文件
匿名集的大小 4
文件： 146 当前是第 0 个点
轨迹第一个停留点， 生成点与原始点距离为： 824.6695960196902
匿名集的大小 4
文件： 146 当前是第 1 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 4
文件： 146 当前是第 2 个点
该轨迹点与前一个点相同
生成点与原始点距离为： 487.38830445873157
匿名集的大小 4
文件： 146 当前是第 3 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 4
文件： 146 当前是第 4 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 4
文件： 146 当前是第 

已写入 st_poi_f 文件
已写入 si_l_f 文件
已写入 si_h_f 文件
匿名集的大小 6
文件： 167 当前是第 0 个点
轨迹第一个停留点， 生成点与原始点距离为： 406.8267862677455
匿名集的大小 6
文件： 167 当前是第 1 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 6
文件： 167 当前是第 2 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 6
文件： 167 当前是第 3 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 6
文件： 167 当前是第 4 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 6
文件： 167 当前是第 5 个点
该轨迹点与前一个点相同
生成点与原始点距离为： 1145.12963286462
匿名集的大小 6
文件： 167 当前是第 6 个点
该轨迹点与前一个点相同
生成点与原始点距离为： 924.9243725602198
匿名集的大小 6
文件： 167 当前是第 7 个点
该轨迹点与前一个点相同
生成点与原始点距离为： 1619.3616126752802
生成点与原始点距离为： 1256.2725300397528
匿名集的大小 6
文件： 167 当前是第 8 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 6
文件： 167 当前是第 9 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 6
文件： 167 当前是第 10 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 6
文件： 167 当前是第 11 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 6
文件： 167 当前是第 12 个点
该轨迹点既不是第一个，与前一个点也不相同
用户在文件:  167 中共有重构位置： 13 个
s_cluster 0_1 r_2 3 2
用户：  0  在文件：  47  中共有停留点有：  14   个
已写入 st_poi_f 文件
已写入 si_l_f 文件
已写入 si_h_f 文件
匿名集的大小 3
文件： 47 当前是第 0 个点
轨迹第一个停留点， 生成点与原始点距离为： 115.78738887892789
匿名集的大小 3
文件： 47 当前是第 1

匿名集的大小 3
文件： 47 当前是第 11 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 47 当前是第 12 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 47 当前是第 13 个点
该轨迹点既不是第一个，与前一个点也不相同
用户在文件:  47 中共有重构位置： 14 个
用户：  0  在文件：  79  中共有停留点有：  10   个
已写入 st_poi_f 文件
已写入 si_l_f 文件
已写入 si_h_f 文件
匿名集的大小 3
文件： 79 当前是第 0 个点
轨迹第一个停留点， 生成点与原始点距离为： 405.0295528000973
匿名集的大小 3
文件： 79 当前是第 1 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 79 当前是第 2 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 79 当前是第 3 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 0
文件： 79 当前是第 4 个点
匿名集的大小 3
文件： 79 当前是第 5 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 79 当前是第 6 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 79 当前是第 7 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 79 当前是第 8 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 79 当前是第 9 个点
该轨迹点既不是第一个，与前一个点也不相同
用户在文件:  79 中共有重构位置： 10 个
用户：  0  在文件：  137  中共有停留点有：  11   个
已写入 st_poi_f 文件
已写入 si_l_f 文件
已写入 si_h_f 文件
匿名集的大小 3
文件： 137 当前是第 0 个点
轨迹第一个停留点， 生成点与原始点距离为： 1383.933688790641
匿名集的大小 3
文件： 137 当前是第 1 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 137 当前是第 2 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 137

已写入 st_poi_f 文件
已写入 si_l_f 文件
已写入 si_h_f 文件
匿名集的大小 3
文件： 146 当前是第 0 个点
轨迹第一个停留点， 生成点与原始点距离为： 101.08972302247186
匿名集的大小 3
文件： 146 当前是第 1 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 146 当前是第 2 个点
该轨迹点与前一个点相同
生成点与原始点距离为： 1383.933688790641
匿名集的大小 3
文件： 146 当前是第 3 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 146 当前是第 4 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 146 当前是第 5 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 146 当前是第 6 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 146 当前是第 7 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 146 当前是第 8 个点
该轨迹点既不是第一个，与前一个点也不相同
用户在文件:  146 中共有重构位置： 9 个
用户：  0  在文件：  167  中共有停留点有：  13   个
已写入 st_poi_f 文件
已写入 si_l_f 文件
已写入 si_h_f 文件
匿名集的大小 3
文件： 167 当前是第 0 个点
轨迹第一个停留点， 生成点与原始点距离为： 2.6621832086467285
匿名集的大小 3
文件： 167 当前是第 1 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 167 当前是第 2 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 167 当前是第 3 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 167 当前是第 4 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 167 当前是第 5 个点
该轨迹点与前一个点相同
生成点与原始点距离为： 559.0903151201134
匿名集的大小 3
文件： 167 当前是第 6 个点
该轨迹点与前一个点相同
生成点与原始点距离为

...

已写入 st_poi_f 文件
已写入 si_l_f 文件
已写入 si_h_f 文件
匿名集的大小 3
文件： 146 当前是第 0 个点
轨迹第一个停留点， 生成点与原始点距离为： 101.08972302247186
匿名集的大小 3
文件： 146 当前是第 1 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 146 当前是第 2 个点
该轨迹点与前一个点相同
生成点与原始点距离为： 1658.3556555872053
生成点与原始点距离为： 487.38830445873157
匿名集的大小 3
文件： 146 当前是第 3 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 146 当前是第 4 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 146 当前是第 5 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 146 当前是第 6 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 146 当前是第 7 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 146 当前是第 8 个点
该轨迹点既不是第一个，与前一个点也不相同
用户在文件:  146 中共有重构位置： 9 个
用户：  0  在文件：  167  中共有停留点有：  13   个
已写入 st_poi_f 文件
已写入 si_l_f 文件
已写入 si_h_f 文件
匿名集的大小 3
文件： 167 当前是第 0 个点
轨迹第一个停留点， 生成点与原始点距离为： 38.60341060655225
匿名集的大小 3
文件： 167 当前是第 1 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 167 当前是第 2 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 167 当前是第 3 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 167 当前是第 4 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 167 当前是第 5 个点
该轨迹点与前一个点相同
生成点与原始点距离为： 1145.12963286462
匿名集的大小 3
文件： 167 当前

用户：  0  在文件：  47  中共有停留点有：  14   个
已写入 st_poi_f 文件
已写入 si_l_f 文件
已写入 si_h_f 文件
匿名集的大小 4
文件： 47 当前是第 0 个点
轨迹第一个停留点， 生成点与原始点距离为： 780.2445072145138
匿名集的大小 4
文件： 47 当前是第 1 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 4
文件： 47 当前是第 2 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 4
文件： 47 当前是第 3 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 4
文件： 47 当前是第 4 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 4
文件： 47 当前是第 5 个点
该轨迹点与前一个点相同
生成点与原始点距离为： 237.71236633030807
匿名集的大小 4
文件： 47 当前是第 6 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 4
文件： 47 当前是第 7 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 4
文件： 47 当前是第 8 个点
该轨迹点与前一个点相同
生成点与原始点距离为： 371.0725038698998
匿名集的大小 4
文件： 47 当前是第 9 个点
该轨迹点与前一个点相同
生成点与原始点距离为： 371.0725038698998
匿名集的大小 4
文件： 47 当前是第 10 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 4
文件： 47 当前是第 11 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 4
文件： 47 当前是第 12 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 4
文件： 47 当前是第 13 个点
该轨迹点既不是第一个，与前一个点也不相同
用户在文件:  47 中共有重构位置： 14 个
用户：  0  在文件：  79  中共有停留点有：  10   个
已写入 st_poi_f 文件
已写入 si_l_f 文件
已写入 si_h_f 文件
匿名集的大小 4
文件： 79 当前是第 0 个点
轨迹第一个停留点， 生成点与原始点距离为： 1688.1044935904242
轨迹第一个

匿名集的大小 6
文件： 47 当前是第 11 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 6
文件： 47 当前是第 12 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 6
文件： 47 当前是第 13 个点
该轨迹点既不是第一个，与前一个点也不相同
用户在文件:  47 中共有重构位置： 14 个
用户：  0  在文件：  79  中共有停留点有：  10   个
已写入 st_poi_f 文件
已写入 si_l_f 文件
已写入 si_h_f 文件
匿名集的大小 6
文件： 79 当前是第 0 个点
轨迹第一个停留点， 生成点与原始点距离为： 1658.3556555872053
轨迹第一个停留点， 生成点与原始点距离为： 1795.1140893004765
轨迹第一个停留点， 生成点与原始点距离为： 1795.1140893004765
轨迹第一个停留点， 生成点与原始点距离为： 1431.6850677244413
匿名集的大小 6
文件： 79 当前是第 1 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 6
文件： 79 当前是第 2 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 6
文件： 79 当前是第 3 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 6
文件： 79 当前是第 4 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 6
文件： 79 当前是第 5 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 6
文件： 79 当前是第 6 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 6
文件： 79 当前是第 7 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 6
文件： 79 当前是第 8 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 6
文件： 79 当前是第 9 个点
该轨迹点既不是第一个，与前一个点也不相同
用户在文件:  79 中共有重构位置： 10 个
用户：  0  在文件：  137  中共有停留点有：  11   个
已写入 st_poi_f 文件
已写入 si_l_f 文件
已写入 si_h_f 文件
匿名集的大小 6
文件： 137 当前是第 0 个点
轨迹第一个停留

匿名集的大小 3
文件： 47 当前是第 11 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 47 当前是第 12 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 47 当前是第 13 个点
该轨迹点既不是第一个，与前一个点也不相同
用户在文件:  47 中共有重构位置： 14 个
用户：  0  在文件：  79  中共有停留点有：  10   个
已写入 st_poi_f 文件
已写入 si_l_f 文件
已写入 si_h_f 文件
匿名集的大小 3
文件： 79 当前是第 0 个点
轨迹第一个停留点， 生成点与原始点距离为： 1431.6850677244413
匿名集的大小 3
文件： 79 当前是第 1 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 79 当前是第 2 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 79 当前是第 3 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 79 当前是第 4 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 79 当前是第 5 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 79 当前是第 6 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 79 当前是第 7 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 79 当前是第 8 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 79 当前是第 9 个点
该轨迹点既不是第一个，与前一个点也不相同
用户在文件:  79 中共有重构位置： 10 个
用户：  0  在文件：  137  中共有停留点有：  11   个
已写入 st_poi_f 文件
已写入 si_l_f 文件
已写入 si_h_f 文件
匿名集的大小 3
文件： 137 当前是第 0 个点
轨迹第一个停留点， 生成点与原始点距离为： 1431.6850677244413
匿名集的大小 3
文件： 137 当前是第 1 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 137 当前是第 2 个点
该轨迹点既不是第一个，与前一

已写入 st_poi_f 文件
已写入 si_l_f 文件
已写入 si_h_f 文件
匿名集的大小 3
文件： 137 当前是第 0 个点
轨迹第一个停留点， 生成点与原始点距离为： 1431.6850677244413
匿名集的大小 3
文件： 137 当前是第 1 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 137 当前是第 2 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 137 当前是第 3 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 137 当前是第 4 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 137 当前是第 5 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 137 当前是第 6 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 137 当前是第 7 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 137 当前是第 8 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 137 当前是第 9 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 137 当前是第 10 个点
该轨迹点既不是第一个，与前一个点也不相同
用户在文件:  137 中共有重构位置： 11 个
用户：  0  在文件：  146  中共有停留点有：  9   个
已写入 st_poi_f 文件
已写入 si_l_f 文件
已写入 si_h_f 文件
匿名集的大小 3
文件： 146 当前是第 0 个点
轨迹第一个停留点， 生成点与原始点距离为： 101.08972302247186
匿名集的大小 3
文件： 146 当前是第 1 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 146 当前是第 2 个点
该轨迹点与前一个点相同
生成点与原始点距离为： 1431.6850677244413
匿名集的大小 3
文件： 146 当前是第 3 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 146 当前是第 4 个点
该轨迹点既不是第一个，与前一个点也不相同
匿名集的大小 3
文件： 146 当前是

In [ ]:
##############################   统计重构过程中的保留点   ####################
## 输入 st_pa   所有停留点  三层list  [user][file][st_p]
## 输出 st_poi_f  五元组list   [id, lat, lng,time, category]

#### 从总停留点 st_pa 提取用户文件的停留点
##   用户： 000；  文件： 48， 80， 138， 147， 168 (j = 168-1)    
##   

big_t = [

[	"s_cluster"	,	"0_1"	,	"k_2"	,	2	,	3	],
[	"s_cluster"	,	"0_1"	,	"k_3"	,	3	,	3	],
[	"s_cluster"	,	"0_1"	,	"k_4"	,	4	,	3	],
[	"s_cluster"	,	"0_1"	,	"k_5"	,	5	,	3	],
[	"s_cluster"	,	"0_1"	,	"k_6"	,	6	,	3	],
[	"s_cluster"	,	"0_1"	,	"r_2"	,	3	,	2	],
[	"s_cluster"	,	"0_1"	,	"r_3"	,	3	,	3	],
[	"s_cluster"	,	"0_1"	,	"r_4"	,	3	,	4	],
[	"s_cluster"	,	"0_1"	,	"r_5"	,	3	,	5	],
[	"s_cluster"	,	"0_1"	,	"r_6"	,	3	,	6	],
[	"s_cluster"	,	"0_2"	,	"k_2"	,	2	,	3	],
[	"s_cluster"	,	"0_2"	,	"k_3"	,	3	,	3	],
[	"s_cluster"	,	"0_2"	,	"k_4"	,	4	,	3	],
[	"s_cluster"	,	"0_2"	,	"k_5"	,	5	,	3	],
[	"s_cluster"	,	"0_2"	,	"k_6"	,	6	,	3	],
[	"s_cluster"	,	"0_2"	,	"r_2"	,	3	,	2	],
[	"s_cluster"	,	"0_2"	,	"r_3"	,	3	,	3	],
[	"s_cluster"	,	"0_2"	,	"r_4"	,	3	,	4	],
[	"s_cluster"	,	"0_2"	,	"r_5"	,	3	,	5	],
[	"s_cluster"	,	"0_2"	,	"r_6"	,	3	,	6	],
[	"s_cluster"	,	"0_3"	,	"k_2"	,	2	,	3	],
[	"s_cluster"	,	"0_3"	,	"k_3"	,	3	,	3	],
[	"s_cluster"	,	"0_3"	,	"k_4"	,	4	,	3	],
[	"s_cluster"	,	"0_3"	,	"k_5"	,	5	,	3	],
[	"s_cluster"	,	"0_3"	,	"k_6"	,	6	,	3	],
[	"s_cluster"	,	"0_3"	,	"r_2"	,	3	,	2	],
[	"s_cluster"	,	"0_3"	,	"r_3"	,	3	,	3	],
[	"s_cluster"	,	"0_3"	,	"r_4"	,	3	,	4	],
[	"s_cluster"	,	"0_3"	,	"r_5"	,	3	,	5	],
[	"s_cluster"	,	"0_3"	,	"r_6"	,	3	,	6	],
[	"k_means"	,	"0_1"	,	"k_2"	,	2	,	3	],
[	"k_means"	,	"0_1"	,	"k_3"	,	3	,	3	],
[	"k_means"	,	"0_1"	,	"k_4"	,	4	,	3	],
[	"k_means"	,	"0_1"	,	"k_5"	,	5	,	3	],
[	"k_means"	,	"0_1"	,	"k_6"	,	6	,	3	],
[	"k_means"	,	"0_1"	,	"r_2"	,	3	,	2	],
[	"k_means"	,	"0_1"	,	"r_3"	,	3	,	3	],
[	"k_means"	,	"0_1"	,	"r_4"	,	3	,	4	],
[	"k_means"	,	"0_1"	,	"r_5"	,	3	,	5	],
[	"k_means"	,	"0_1"	,	"r_6"	,	3	,	6	],
[	"k_means"	,	"0_2"	,	"k_2"	,	2	,	3	],
[	"k_means"	,	"0_2"	,	"k_3"	,	3	,	3	],
[	"k_means"	,	"0_2"	,	"k_4"	,	4	,	3	],
[	"k_means"	,	"0_2"	,	"k_5"	,	5	,	3	],
[	"k_means"	,	"0_2"	,	"k_6"	,	6	,	3	],
[	"k_means"	,	"0_2"	,	"r_2"	,	3	,	2	],
[	"k_means"	,	"0_2"	,	"r_3"	,	3	,	3	],
[	"k_means"	,	"0_2"	,	"r_4"	,	3	,	4	],
[	"k_means"	,	"0_2"	,	"r_5"	,	3	,	5	],
[	"k_means"	,	"0_2"	,	"r_6"	,	3	,	6	],
[	"k_means"	,	"0_3"	,	"k_2"	,	2	,	3	],
[	"k_means"	,	"0_3"	,	"k_3"	,	3	,	3	],
[	"k_means"	,	"0_3"	,	"k_4"	,	4	,	3	],
[	"k_means"	,	"0_3"	,	"k_5"	,	5	,	3	],
[	"k_means"	,	"0_3"	,	"k_6"	,	6	,	3	],
[	"k_means"	,	"0_3"	,	"r_2"	,	3	,	2	],
[	"k_means"	,	"0_3"	,	"r_3"	,	3	,	3	],
[	"k_means"	,	"0_3"	,	"r_4"	,	3	,	4	],
[	"k_means"	,	"0_3"	,	"r_5"	,	3	,	5	],
[	"k_means"	,	"0_3"	,	"r_6"	,	3	,	6	],
[	"a_cluster"	,	"0_1"	,	"k_2"	,	2	,	3	],
[	"a_cluster"	,	"0_1"	,	"k_3"	,	3	,	3	],
[	"a_cluster"	,	"0_1"	,	"k_4"	,	4	,	3	],
[	"a_cluster"	,	"0_1"	,	"k_5"	,	5	,	3	],
[	"a_cluster"	,	"0_1"	,	"k_6"	,	6	,	3	],
[	"a_cluster"	,	"0_1"	,	"r_2"	,	3	,	2	],
[	"a_cluster"	,	"0_1"	,	"r_3"	,	3	,	3	],
[	"a_cluster"	,	"0_1"	,	"r_4"	,	3	,	4	],
[	"a_cluster"	,	"0_1"	,	"r_5"	,	3	,	5	],
[	"a_cluster"	,	"0_1"	,	"r_6"	,	3	,	6	],
[	"a_cluster"	,	"0_2"	,	"k_2"	,	2	,	3	],
[	"a_cluster"	,	"0_2"	,	"k_3"	,	3	,	3	],
[	"a_cluster"	,	"0_2"	,	"k_4"	,	4	,	3	],
[	"a_cluster"	,	"0_2"	,	"k_5"	,	5	,	3	],
[	"a_cluster"	,	"0_2"	,	"k_6"	,	6	,	3	],
[	"a_cluster"	,	"0_2"	,	"r_2"	,	3	,	2	],
[	"a_cluster"	,	"0_2"	,	"r_3"	,	3	,	3	],
[	"a_cluster"	,	"0_2"	,	"r_4"	,	3	,	4	],
[	"a_cluster"	,	"0_2"	,	"r_5"	,	3	,	5	],
[	"a_cluster"	,	"0_2"	,	"r_6"	,	3	,	6	],
[	"a_cluster"	,	"0_3"	,	"k_2"	,	2	,	3	],
[	"a_cluster"	,	"0_3"	,	"k_3"	,	3	,	3	],
[	"a_cluster"	,	"0_3"	,	"k_4"	,	4	,	3	],
[	"a_cluster"	,	"0_3"	,	"k_5"	,	5	,	3	],
[	"a_cluster"	,	"0_3"	,	"k_6"	,	6	,	3	],
[	"a_cluster"	,	"0_3"	,	"r_2"	,	3	,	2	],
[	"a_cluster"	,	"0_3"	,	"r_3"	,	3	,	3	],
[	"a_cluster"	,	"0_3"	,	"r_4"	,	3	,	4	],
[	"a_cluster"	,	"0_3"	,	"r_5"	,	3	,	5	],
[	"a_cluster"	,	"0_3"	,	"r_6"	,	3	,	6	],
[	"mb_kmeans"	,	"0_1"	,	"k_2"	,	2	,	3	],
[	"mb_kmeans"	,	"0_1"	,	"k_3"	,	3	,	3	],
[	"mb_kmeans"	,	"0_1"	,	"k_4"	,	4	,	3	],
[	"mb_kmeans"	,	"0_1"	,	"k_5"	,	5	,	3	],
[	"mb_kmeans"	,	"0_1"	,	"k_6"	,	6	,	3	],
[	"mb_kmeans"	,	"0_1"	,	"r_2"	,	3	,	2	],
[	"mb_kmeans"	,	"0_1"	,	"r_3"	,	3	,	3	],
[	"mb_kmeans"	,	"0_1"	,	"r_4"	,	3	,	4	],
[	"mb_kmeans"	,	"0_1"	,	"r_5"	,	3	,	5	],
[	"mb_kmeans"	,	"0_1"	,	"r_6"	,	3	,	6	],
[	"mb_kmeans"	,	"0_2"	,	"k_2"	,	2	,	3	],
[	"mb_kmeans"	,	"0_2"	,	"k_3"	,	3	,	3	],
[	"mb_kmeans"	,	"0_2"	,	"k_4"	,	4	,	3	],
[	"mb_kmeans"	,	"0_2"	,	"k_5"	,	5	,	3	],
[	"mb_kmeans"	,	"0_2"	,	"k_6"	,	6	,	3	],
[	"mb_kmeans"	,	"0_2"	,	"r_2"	,	3	,	2	],
[	"mb_kmeans"	,	"0_2"	,	"r_3"	,	3	,	3	],
[	"mb_kmeans"	,	"0_2"	,	"r_4"	,	3	,	4	],
[	"mb_kmeans"	,	"0_2"	,	"r_5"	,	3	,	5	],
[	"mb_kmeans"	,	"0_2"	,	"r_6"	,	3	,	6	],
[	"mb_kmeans"	,	"0_3"	,	"k_2"	,	2	,	3	],
[	"mb_kmeans"	,	"0_3"	,	"k_3"	,	3	,	3	],
[	"mb_kmeans"	,	"0_3"	,	"k_4"	,	4	,	3	],
[	"mb_kmeans"	,	"0_3"	,	"k_5"	,	5	,	3	],
[	"mb_kmeans"	,	"0_3"	,	"k_6"	,	6	,	3	],
[	"mb_kmeans"	,	"0_3"	,	"r_2"	,	3	,	2	],
[	"mb_kmeans"	,	"0_3"	,	"r_3"	,	3	,	3	],
[	"mb_kmeans"	,	"0_3"	,	"r_4"	,	3	,	4	],
[	"mb_kmeans"	,	"0_3"	,	"r_5"	,	3	,	5	],
[	"mb_kmeans"	,	"0_3"	,	"r_6"	,	3	,	6	]

  
]

"""kd = 2        # k 匿名度
o_r = 3     #  最高高度
o_l = 1     #  最低高度
cluster = "mb_kmeans"    # 聚类器
ex_times = "0_1"      #第几次实验
at = "k_2"      # 参数设置
i = 0
"""
a111 = 0
a121 = 0
a131 = 0
a141 = 0
a151 = 0
a211 = 0
a221 = 0
a231 = 0
a241 = 0
a251 = 0
a311 = 0
a321 = 0
a331 = 0
a341 = 0
a351 = 0
a411 = 0
a421 = 0
a431 = 0
a441 = 0
a451 = 0

b111 = 0
b121 = 0
b131 = 0
b141 = 0
b151 = 0
b211 = 0
b221 = 0
b231 = 0
b241 = 0
b251 = 0
b311 = 0
b321 = 0
b331 = 0
b341 = 0
b351 = 0
b411 = 0
b421 = 0
b431 = 0
b441 = 0
b451 = 0


a112 = 0
a122 = 0
a132 = 0
a142 = 0
a152 = 0
a212 = 0
a222 = 0
a232 = 0
a242 = 0
a252 = 0
a312 = 0
a322 = 0
a332 = 0
a342 = 0
a352 = 0
a412 = 0
a422 = 0
a432 = 0
a442 = 0
a452 = 0


b112 = 0
b122 = 0
b132 = 0
b142 = 0
b152 = 0
b212 = 0
b222 = 0
b232 = 0
b242 = 0
b252 = 0
b312 = 0
b322 = 0
b332 = 0
b342 = 0
b352 = 0
b412 = 0
b422 = 0
b432 = 0
b442 = 0
b452 = 0


a113 = 0
a123 = 0
a133 = 0
a143 = 0
a153 = 0
a213 = 0
a223 = 0
a233 = 0
a243 = 0
a253 = 0
a313 = 0
a323 = 0
a333 = 0
a343 = 0
a353 = 0
a413 = 0
a423 = 0
a433 = 0
a443 = 0
a453 = 0

b113 = 0
b123 = 0
b133 = 0
b143 = 0
b153 = 0
b213 = 0
b223 = 0
b233 = 0
b243 = 0
b253 = 0
b313 = 0
b323 = 0
b333 = 0
b343 = 0
b353 = 0
b413 = 0
b423 = 0
b433 = 0
b443 = 0
b453 = 0

a114 = 0
a124 = 0
a134 = 0
a144 = 0
a154 = 0
a214 = 0
a224 = 0
a234 = 0
a244 = 0
a254 = 0
a314 = 0
a324 = 0
a334 = 0
a344 = 0
a354 = 0
a414 = 0
a424 = 0
a434 = 0
a444 = 0
a454 = 0

b114 = 0
b124 = 0
b134 = 0
b144 = 0
b154 = 0
b214 = 0
b224 = 0
b234 = 0
b244 = 0
b254 = 0
b314 = 0
b324 = 0
b334 = 0
b344 = 0
b354 = 0
b414 = 0
b424 = 0
b434 = 0
b444 = 0
b454 = 0






i = 0
o_l = 1     #  最低高度
num = 0
num_tot = 0

for bb in big_t:
    cluster = bb[0]   # 聚类器  a_cluster,,mb_kmeans
    ex_times = bb[1]      #第几次实验
    at = bb[2]         # 变化的参数
    kd = bb[3]        # k 匿名度
    o_r = bb[4]    #  最高高度
    print(cluster, ex_times, at,kd,o_r)
    
    ########## semantic_tree    ###############
    filename = ".\\os_file\\st_poi_clusters\\"+str(cluster)+"\\semantic_tree_8.pkl"
    st = read_q(filename)

    wz = [47,79,137,146,167]
    for aa in wz:
        j = aa
    #j = 167   # 第48个文件  48   138   168

        filename1 = ".\\os_file\\res_8\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\st_poi_f_"+str(i)+"_"+str(j)+".pkl"
        filename2 = ".\\os_file\\res_8\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\si_l_f_"+str(i)+"_"+str(j)+".pkl"
        filename3 = ".\\os_file\\res_8\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\si_h_f_"+str(i)+"_"+str(j)+".pkl"
        filename4 = ".\\os_file\\res_8\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\k_as_"+str(i)+"_"+str(j)+".pkl" 
        filename5 = ".\\os_file\\res_8\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\rec_tra_"+str(i)+"_"+str(j)+".pkl"
        filename6 = ".\\os_file\\res_8\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\rec_cos_"+str(i)+"_"+str(j)+".pkl"
        filename7 = ".\\os_file\\res_8\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\tra_leaf_"+str(i)+"_"+str(j)+".pkl"
        filename8 = ".\\os_file\\res_8\\ori_p_num.pkl"

#############################################################
#  输入  st_pa 所有停留点   三重列表
#  输出  st_pa_f  文件的停留点    一维列表

        st_pa_f = []
#st_p = []    #  停留点的list 

        m = st_pa

        if len(m[i]) == 0:            #  如果  user i 没有文件 
            print("该用户没有轨迹文件")
            sys.exit()
        else:       
            if len(m[i][j]) == 0:       #  文件内没有stay point，
                print("该文件内没有停留点")
                sys.exit()
            else:      #  文件内有 stay point               #  user_file_st-p
                for k in range(0, len(m[i][j])):
                            # 对用户 i 文件 j, 停留点 k, st_p[i][j][k]     停留点
                    a = []
                    a = m[i][j][k]
                    st_pa_f.append(a)
            
        b = 0
        b = len(st_pa_f)
        num_tot += b
        print("用户： ",i," 在文件： ",j," 中共有停留点有： ", b,"  个")

#################  匹配 st_pa_f ----> st_poi_f ############### 
############   把 文件停留点， 经 st_poi_dic 匹配, 得 POI+ category： st_poi_f
# 输入  文件停留点st_pa_f,    st_poi_dic   停留点与POI匹配的字典列表
# 输出  st_poi_f      文件与POI匹配列表   二维列表
# [['无尽', 40.005973, 116.32533400000001, 65.81666663987562, 'scenic'],

        st_poi_f = []
        for l in range(0, len(st_pa_f)):
#for i in range(0, 20):
    
            st_p_m = match_1(st_pa_f[l], st_poi_dic)
            st_poi_f.append(st_p_m)

        write_q(st_poi_f, filename1)
        print("已写入 st_poi_f 文件")

##############################  semantic importance  ##################
## 输入 st_poi_f  五元组-两维list   [id, lat, lng,time, category] ;    st_poi_pro_p, st_poi_pro
## 输出 si_l_f    六元组-两维list   [id, lat, lng,time, category,level]
# 输出文件 si_l_f_0_47.pkl   

        f = st_poi_f
        pro_p = st_poi_pro_p
        pro = st_poi_pro

        si_l_f = []
        for k in range(len(f)):
    
            ca = f[k][4]
    
            flag = 0
    
            for item1 in pro:
        
                if flag == 1:
                    break
                else:
            
                    for item2 in pro[item1]:
                        if flag == 1:
                            break
                        else:
                            if ca == item2:         # 判断语义类型匹配
                                flag = 1
                                a = 0
                                a = pro[item1][item2]
                                b = 0
                                b = pro_p[item1][item2]
                        
                                if a == 0:
                                    print("st_poi_pro 没有该类型的停留点")
                            
                                c=0
                                c = b * math.log(1/a)
                        
                                d = [f[k][0],f[k][1],f[k][2],f[k][3],f[k][4],c]
                                si_l_f.append(d)
                    
        write_q(si_l_f, filename2)
        print("已写入 si_l_f 文件")
###########################  si_h_f 语义树高度   #######################
## 输入 si_l_f， o_l, o_r   
## 输出 si_h_f     六元组-两维list   [id, lat, lng,time, category,height]
                  # 输出文件 si_h_f_0_47.pkl 

        al = []
        for k in range(len(si_l_f)):
            al.append(si_l_f[k][5])
        w = max(al)

        si_h_f = []
        for k in range(len(si_l_f)):
            a = si_l_f[k][5]
            b = a/w
            c = 1-b
    
            l = b * o_l + c *o_r
            h = round(l)
    
            d = []
            d = [si_l_f[k][0],si_l_f[k][1],si_l_f[k][2],si_l_f[k][3],si_l_f[k][4],h]
            si_h_f.append(d)

        write_q(si_h_f, filename3)
        print("已写入 si_h_f 文件")
#####################  轨迹重构 rec_tra  rec_tra_i_j.pkl ################
# 输入  si_h_f 文件内停留点的语义高度； st 语义树；
# 重构轨迹 rec_tra  

        si = si_h_f    # 文件中停留点list 6元组
        st_l = st.leaves()   # st 叶子结点list

        rec_tra = []   # 存储 重构的轨迹
        k_as = []      # 存储 K 匿名集
        rec_cos = []    #  角度
        tra_leaf = []   # 存储匹配的原始叶子结点
        for k in range(len(si)):
    
            h = si[k][5]      # 语义匿名高度
            n_id = match_2(si[k], st_l)     # 文件停留点与叶子结点匹配，   返回匹配结点  id
            sp_node = st.get_node(n_id)
            tra_leaf.append(sp_node)
    #print(st.level(n_id)+1, h)
            m_id = n_id
            if st.level(m_id) < h:
                m_id = "poi"
            else:
                for l in range(h):
            #print(l)
                    m_id = st.parent(m_id).identifier   # 返回 中间结点 id
    
    #print(m_id)
            m_l = st.leaves(m_id)    # 返回 中间结点 对应的 叶子结点  匿名集  [node, node, node]
    ##############  删除 空结点  #############
            ai = 0
            while ai < len(m_l):     #  删除空叶子结点
        
                if m_l[ai].data == None:
                    del m_l[ai]
                else:
                    ai += 1
   
    ########################################
            for l in range(len(m_l)):         ## staying 特征概率化
                abc = m_l[l].data["staying"]
                bcd = proba(abc)
                m_l[l].data["staying"] = bcd
    ########################### 选点  ##############################   
    #  对轨迹第一个点，生成点只要在1500范围内即可
    #  对轨迹中连续的相同点，条件同上
    #  对轨迹中非同一点，角要小于 53 度
            k_as_m = gen_k_as(si[k], m_l, kd)       # 生成K-匿名集，  [node,node,node]
            print("匿名集的大小", len(k_as_m))
            #print("已生成 si_h_f:")
            t2 = [si[k][1],si[k][2]]  
            print("文件：",j,"当前是第",k,"个点")
            
            if len(k_as_m) == 0:         #  如果匿名集为空，返回原结点
                recon_node = sp_node
                num += 1
                if cluster == "k_means":
                    if at == "k_2":    a111 += 1
                    elif at == "k_3":  a121 += 1
                    elif at == "k_4":  a131 += 1
                    elif at == "k_5":  a141 += 1
                    elif at == "k_6":  a151 += 1
                    elif at == "r_2":  b111 += 1
                    elif at == "r_3":  b121 += 1
                    elif at == "r_4":  b131 += 1
                    elif at == "r_5":  b141 += 1
                    else:              b151 += 1
                elif cluster == "mb_kmeans":
                    if at == "k_2":    a211 += 1
                    elif at == "k_3":  a221 += 1
                    elif at == "k_4":  a231 += 1
                    elif at == "k_5":  a241 += 1
                    elif at == "k_6":  a251 += 1
                    elif at == "r_2":  b211 += 1
                    elif at == "r_3":  b221 += 1
                    elif at == "r_4":  b231 += 1
                    elif at == "r_5":  b241 += 1
                    else:              b251 += 1
                elif cluster == "a_cluster":
                    if at == "k_2":    a311 += 1
                    elif at == "k_3":  a321 += 1
                    elif at == "k_4":  a331 += 1
                    elif at == "k_5":  a341 += 1
                    elif at == "k_6":  a351 += 1
                    elif at == "r_2":  b311 += 1
                    elif at == "r_3":  b321 += 1
                    elif at == "r_4":  b331 += 1
                    elif at == "r_5":  b341 += 1
                    else:              b351 += 1
                else:
                    if at == "k_2":    a411 += 1
                    elif at == "k_3":  a421 += 1
                    elif at == "k_4":  a431 += 1
                    elif at == "k_5":  a441 += 1
                    elif at == "k_6":  a451 += 1
                    elif at == "r_2":  b411 += 1
                    elif at == "r_3":  b421 += 1
                    elif at == "r_4":  b431 += 1
                    elif at == "r_5":  b441 += 1
                    else:              b451 += 1
                
                #num += 1
            else:
                
                if k == 0:               # 对轨迹第一个结点，要求在 1.5km 内生成 fake 结点
                    flag = 0
                    for aa in k_as_m:               
                        r2 = [aa.data["location"]["lat"],aa.data["location"]["lng"]]
                        dis = Dis([t2, r2])
                        aa.data["dis"] = dis
                        if dis < 1500:
                            flag = 1
                            
                    if flag == 0:                              # 判断是否存在 1.5km 内的点
                        recon_node = sp_node
                        num += 1
                        if cluster == "k_means":
                            if at == "k_2":    a112 += 1
                            elif at == "k_3":  a122 += 1
                            elif at == "k_4":  a132 += 1
                            elif at == "k_5":  a142 += 1
                            elif at == "k_6":  a152 += 1
                            elif at == "r_2":  b112 += 1
                            elif at == "r_3":  b122 += 1
                            elif at == "r_4":  b132 += 1
                            elif at == "r_5":  b142 += 1
                            else:              b152 += 1
                        elif cluster == "mb_kmeans":
                            if at == "k_2":    a212 += 1
                            elif at == "k_3":  a222 += 1
                            elif at == "k_4":  a232 += 1
                            elif at == "k_5":  a242 += 1
                            elif at == "k_6":  a252 += 1
                            elif at == "r_2":  b212 += 1
                            elif at == "r_3":  b222 += 1
                            elif at == "r_4":  b232 += 1
                            elif at == "r_5":  b242 += 1
                            else:              b252 += 1
                        elif cluster == "a_cluster":
                            if at == "k_2":    a312 += 1
                            elif at == "k_3":  a322 += 1
                            elif at == "k_4":  a332 += 1
                            elif at == "k_5":  a342 += 1
                            elif at == "k_6":  a352 += 1
                            elif at == "r_2":  b312 += 1
                            elif at == "r_3":  b322 += 1
                            elif at == "r_4":  b332 += 1
                            elif at == "r_5":  b342 += 1
                            else:              b352 += 1
                        else:
                            if at == "k_2":    a412 += 1
                            elif at == "k_3":  a422 += 1
                            elif at == "k_4":  a432 += 1
                            elif at == "k_5":  a442 += 1
                            elif at == "k_6":  a452 += 1
                            elif at == "r_2":  b412 += 1
                            elif at == "r_3":  b422 += 1
                            elif at == "r_4":  b432 += 1
                            elif at == "r_5":  b442 += 1
                            else:              b452 += 1
                        
                        #num += 1
                    else:
                        flag = 0
                        while flag == 0:
                            idx = np.random.randint(len(k_as_m))      # k范围内，随机生成索引
                            dis = k_as_m[idx].data["dis"]
                            print("轨迹第一个停留点， 生成点与原始点距离为：", dis)
                            if dis < 1500:
                                flag = 1
                        recon_node = k_as_m[idx]
                else:
                    t1 = [si[k-1][1],si[k-1][2]]
                    t1_t2 = Dis([t1,t2])
            
                    if t1_t2 == 0:         #  如果是同一个停留点，则只要距离不超过  1500 米就行
                        print("该轨迹点与前一个点相同")
                        flag = 0
                        for aa in k_as_m:               
                            r2 = [aa.data["location"]["lat"],aa.data["location"]["lng"]]
                            dis = Dis([t2, r2])
                            aa.data["dis"] = dis
                            if dis < 1500:
                                flag = 1
                            
                        if flag == 0:                              # 判断是否存在 1.5km 内的点
                            recon_node = sp_node
                            num += 1
                            if cluster == "k_means":
                                if at == "k_2":    a113 += 1
                                elif at == "k_3":  a123 += 1
                                elif at == "k_4":  a133 += 1
                                elif at == "k_5":  a143 += 1
                                elif at == "k_6":  a153 += 1
                                elif at == "r_2":  b113 += 1
                                elif at == "r_3":  b123 += 1
                                elif at == "r_4":  b133 += 1
                                elif at == "r_5":  b143 += 1
                                else:              b153 += 1
                            elif cluster == "mb_kmeans":
                                if at == "k_2":    a213 += 1
                                elif at == "k_3":  a223 += 1
                                elif at == "k_4":  a233 += 1
                                elif at == "k_5":  a243 += 1
                                elif at == "k_6":  a253 += 1
                                elif at == "r_2":  b213 += 1
                                elif at == "r_3":  b223 += 1
                                elif at == "r_4":  b233 += 1
                                elif at == "r_5":  b243 += 1
                                else:              b253 += 1
                            elif cluster == "a_cluster":
                                if at == "k_2":    a313 += 1
                                elif at == "k_3":  a323 += 1
                                elif at == "k_4":  a333 += 1
                                elif at == "k_5":  a343 += 1
                                elif at == "k_6":  a353 += 1
                                elif at == "r_2":  b313 += 1
                                elif at == "r_3":  b323 += 1
                                elif at == "r_4":  b333 += 1
                                elif at == "r_5":  b343 += 1
                                else:              b353 += 1
                            else:
                                if at == "k_2":    a413 += 1
                                elif at == "k_3":  a423 += 1
                                elif at == "k_4":  a433 += 1
                                elif at == "k_5":  a443 += 1
                                elif at == "k_6":  a453 += 1
                                elif at == "r_2":  b413 += 1
                                elif at == "r_3":  b423 += 1
                                elif at == "r_4":  b433 += 1
                                elif at == "r_5":  b443 += 1
                                else:              b453 += 1
                            
                            #num += 1
                        else:
                            flag = 0
                            while flag == 0:
                                idx = np.random.randint(len(k_as_m))      # k范围内，随机生成索引
                                dis = k_as_m[idx].data["dis"]
                                print("生成点与原始点距离为：", dis)
                                if dis < 1500:
                                    flag = 1
                            recon_node = k_as_m[idx]
                        """
                        flag = 0
                        while flag == 0:
                            idx = np.random.randint(len(k_as_m))      # k范围内，随机生成索引
                            r2 = [k_as_m[idx].data["location"]["lat"],k_as_m[idx].data["location"]["lng"]]
                    
                            t2_r2 = Dis([t2,r2])
                            print("该轨迹点与生成点距离为： ", t2_r2)
                            if t2_r2 < 1500:
                                flag = 1
                        recon_node = k_as_m[idx]  """
                    else:
                
                        print("该轨迹点既不是第一个，与前一个点也不相同")
                        flag = 0
                        for aa in k_as_m:
                            r2 = [aa.data["location"]["lat"],aa.data["location"]["lng"]]
                            t1_r2 = Dis([t1,r2])
                            t2_r2 = Dis([t2,r2])
                            if t1_r2 == 0:     #       如果 fake 位置与前一个位置相同，则放弃
                                aa.data["cos"] = -1
                            else:                   #   正常结点， 在90度偏移范围内选择 fake 结点 
                                dista = [t1_t2, t1_r2, t2_r2]
                                ttcos = tra_cos(dista)
                                aa.data["cos"] = ttcos
                                if ttcos >= 0:
                                    flag = 1
                    
                        if flag == 1:
                            tcos = -1
                            while tcos < 0:
                                idx = np.random.randint(len(k_as_m))      # k范围内，随机生成索引
                                tcos = k_as_m[idx].data["cos"]
                        #print("当前生成点与原始轨迹两个点的偏角余弦为：",tcos)
                            recon_node = k_as_m[idx]
                        else:
                            tcos = 1
                            sp_node.data["cos"] = tcos
                            recon_node = sp_node
                            num += 1
                            if cluster == "k_means":
                                if at == "k_2":    a114 += 1
                                elif at == "k_3":  a124 += 1
                                elif at == "k_4":  a134 += 1
                                elif at == "k_5":  a144 += 1
                                elif at == "k_6":  a154 += 1
                                elif at == "r_2":  b114 += 1
                                elif at == "r_3":  b124 += 1
                                elif at == "r_4":  b134 += 1
                                elif at == "r_5":  b144 += 1
                                else:              b154 += 1
                            elif cluster == "mb_kmeans":
                                if at == "k_2":    a214 += 1
                                elif at == "k_3":  a224 += 1
                                elif at == "k_4":  a234 += 1
                                elif at == "k_5":  a244 += 1
                                elif at == "k_6":  a254 += 1
                                elif at == "r_2":  b214 += 1
                                elif at == "r_3":  b224 += 1
                                elif at == "r_4":  b234 += 1
                                elif at == "r_5":  b244 += 1
                                else:              b254 += 1
                            elif cluster == "a_cluster":
                                if at == "k_2":    a314 += 1
                                elif at == "k_3":  a324 += 1
                                elif at == "k_4":  a334 += 1
                                elif at == "k_5":  a344 += 1
                                elif at == "k_6":  a354 += 1
                                elif at == "r_2":  b314 += 1
                                elif at == "r_3":  b324 += 1
                                elif at == "r_4":  b334 += 1
                                elif at == "r_5":  b344 += 1
                                else:              b354 += 1
                            else:
                                if at == "k_2":    a414 += 1
                                elif at == "k_3":  a424 += 1
                                elif at == "k_4":  a434 += 1
                                elif at == "k_5":  a444 += 1
                                elif at == "k_6":  a454 += 1
                                elif at == "r_2":  b414 += 1
                                elif at == "r_3":  b424 += 1
                                elif at == "r_4":  b434 += 1
                                elif at == "r_5":  b444 += 1
                                else:              b454 += 1
                            
                             
                        
                        rec_cos.append(tcos)

            rec_tra.append(recon_node)
            k_as.append(k_as_m)
        
        write_q(tra_leaf, filename7)
        write_q(rec_tra, filename5)
        write_q(k_as,filename4)
        write_q(rec_cos,filename6)
        print("用户在文件: ",j,"中共有重构位置：",len(rec_tra),"个")
        
a11 = [a111, a112, a113, a114]
a12 = [a121, a122, a123, a124]
a13 = [a131, a132, a133, a134]
a14 = [a141, a142, a143, a144]
a15 = [a151, a152, a153, a154]

b11 = [b111, b112, b113, b114]
b12 = [b121, b122, b123, b124]
b13 = [b131, b132, b133, b134]
b14 = [b141, b142, b143, b144]
b15 = [b151, b152, b153, b154]


a21 = [a211, a212, a213, a214]
a22 = [a221, a222, a223, a224]
a23 = [a231, a232, a233, a234]
a24 = [a241, a242, a243, a244]
a25 = [a251, a252, a253, a254]

b21 = [b211, b212, b213, b214]
b22 = [b221, b222, b223, b224]
b23 = [b231, b232, b233, b234]
b24 = [b241, b242, b243, b244]
b25 = [b251, b252, b253, b254]


a31 = [a311, a312, a313, a314]
a32 = [a321, a322, a323, a324]
a33 = [a331, a332, a333, a334]
a34 = [a341, a342, a343, a344]
a35 = [a351, a352, a353, a354]

b31 = [b311, b312, b313, b314]
b32 = [b321, b322, b323, b324]
b33 = [b331, b332, b333, b334]
b34 = [b341, b342, b343, b344]
b35 = [b351, b352, b353, b354]



a41 = [a411, a412, a413, a414]
a42 = [a421, a422, a423, a424]
a43 = [a431, a432, a433, a434]
a44 = [a441, a442, a443, a444]
a45 = [a451, a452, a453, a454]

b41 = [b411, b412, b413, b414]
b42 = [b421, b422, b423, b424]
b43 = [b431, b432, b433, b434]
b44 = [b441, b442, b443, b444]
b45 = [b451, b452, b453, b454]


        
a1 = [a11,a12,a13,a14,a15]
a2 = [a21,a22,a23,a24,a25]
a3 = [a31,a32,a33,a34,a35]
a4 = [a41,a42,a43,a44,a45]
a = [a1,a2,a3,a4]

b1 = [b11,b12,b13,b14,b15]
b2 = [b21,b22,b23,b24,b25]
b3 = [b31,b32,b33,b34,b35]
b4 = [b41,b42,b43,b44,b45]
b = [b1,b2,b3,b4]
ori_p_num = [a,b] 
write_q(ori_p_num, filename8)
print("在为用户重构轨迹过程中共保留原始结点：",num,"个；重构结点总数为：",num_tot,"个；比例为：",round(num/num_tot,4))

In [8]:
b = 2
c = 0
a = [b,c]
#b = 2
#a[0].append(b)
a

[2, 0]

In [5]:
a[0]

[1, 2]

In [26]:
filename1 = ".\\os_file\\res\\"+"k_means"+"\\"+"0_1"+"\\"+"k_2"+ "\\st_poi_f_"+"0"+"_"+"47"+".pkl"
filename2 = ".\\os_file\\res\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\si_l_f_"+str(i)+"_"+str(j)+".pkl"
filename3 = ".\\os_file\\res\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\si_h_f_"+str(i)+"_"+str(j)+".pkl"
filename4 = ".\\os_file\\res\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\k_as_"+str(i)+"_"+str(j)+".pkl" 
filename5 = ".\\os_file\\res\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\rec_tra_"+str(i)+"_"+str(j)+".pkl"
filename6 = ".\\os_file\\res\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\rec_cos_"+str(i)+"_"+str(j)+".pkl"
filename7 = ".\\os_file\\res\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\tra_leaf_"+str(i)+"_"+str(j)+".pkl"


NameError: name 'j' is not defined

In [19]:
filename1 = ".\\os_file\\res\\"+"k_means"+"\\"+"0_1"+"\\"+"k_2"+ "\\tra_leaf_"+"0"+"_"+"47"+".pkl"
rec_cos = read_q(filename1)
rec_cos

FileNotFoundError: [Errno 2] No such file or directory: '.\\os_file\\res\\k_means\\0_1\\k_2\\tra_leaf_0_47.pkl'

In [9]:
for aa in k_as[0]:
    print(aa)

Node(tag=enter_scenic_0_1_0, identifier=enter_scenic_0_1_0, data={'id': '无尽', 'location': {'lat': 40.005973, 'lng': 116.32533400000001}, 'staying': [0, 0, 0, 0, 0, 0, 0, 0, 1.0], 'category': 'scenic', 'distance': 0.0})
Node(tag=enter_scenic_0_1_21, identifier=enter_scenic_0_1_21, data={'id': '零零阁', 'location': {'lat': 40.006861, 'lng': 116.326044}, 'staying': [0, 0, 0, 0, 0, 0, 0, 0, 1.0], 'category': 'scenic', 'distance': 115.78731618244935})


In [25]:
big_t = [
    [	"s_cluster"	,	"0_1"	,	"k_2"	,	2	,	3	],
[	"s_cluster"	,	"0_1"	,	"k_3"	,	3	,	3	],
[	"s_cluster"	,	"0_1"	,	"k_4"	,	4	,	3	],
[	"s_cluster"	,	"0_1"	,	"k_5"	,	5	,	3	],
[	"s_cluster"	,	"0_1"	,	"k_6"	,	6	,	3	],
[	"s_cluster"	,	"0_1"	,	"r_2"	,	3	,	2	],
[	"s_cluster"	,	"0_1"	,	"r_3"	,	3	,	3	],
[	"s_cluster"	,	"0_1"	,	"r_4"	,	3	,	4	],
[	"s_cluster"	,	"0_1"	,	"r_5"	,	3	,	5	],
[	"s_cluster"	,	"0_1"	,	"r_6"	,	3	,	6	],
[	"s_cluster"	,	"0_2"	,	"k_2"	,	2	,	3	],
[	"s_cluster"	,	"0_2"	,	"k_3"	,	3	,	3	],
[	"s_cluster"	,	"0_2"	,	"k_4"	,	4	,	3	],
[	"s_cluster"	,	"0_2"	,	"k_5"	,	5	,	3	],
[	"s_cluster"	,	"0_2"	,	"k_6"	,	6	,	3	],
[	"s_cluster"	,	"0_2"	,	"r_2"	,	3	,	2	],
[	"s_cluster"	,	"0_2"	,	"r_3"	,	3	,	3	],
[	"s_cluster"	,	"0_2"	,	"r_4"	,	3	,	4	],
[	"s_cluster"	,	"0_2"	,	"r_5"	,	3	,	5	],
[	"s_cluster"	,	"0_2"	,	"r_6"	,	3	,	6	],
[	"s_cluster"	,	"0_3"	,	"k_2"	,	2	,	3	],
[	"s_cluster"	,	"0_3"	,	"k_3"	,	3	,	3	],
[	"s_cluster"	,	"0_3"	,	"k_4"	,	4	,	3	],
[	"s_cluster"	,	"0_3"	,	"k_5"	,	5	,	3	],
[	"s_cluster"	,	"0_3"	,	"k_6"	,	6	,	3	],
[	"s_cluster"	,	"0_3"	,	"r_2"	,	3	,	2	],
[	"s_cluster"	,	"0_3"	,	"r_3"	,	3	,	3	],
[	"s_cluster"	,	"0_3"	,	"r_4"	,	3	,	4	],
[	"s_cluster"	,	"0_3"	,	"r_5"	,	3	,	5	],
[	"s_cluster"	,	"0_3"	,	"r_6"	,	3	,	6	]
]

i = 0
o_l = 1     #  最低高度

for bb in big_t:
    cluster = bb[0]   # 聚类器  a_cluster,,mb_kmeans
    ex_times = bb[1]      #第几次实验
    at = bb[2]
    kd = bb[3]        # k 匿名度
    o_r = bb[4]    #  最高高度
    print(cluster, ex_times, at,kd,o_r)

s_cluster 0_1 k_2 2 3
s_cluster 0_1 k_3 3 3
s_cluster 0_1 k_4 4 3
s_cluster 0_1 k_5 5 3
s_cluster 0_1 k_6 6 3
s_cluster 0_1 r_2 3 2
s_cluster 0_1 r_3 3 3
s_cluster 0_1 r_4 3 4
s_cluster 0_1 r_5 3 5
s_cluster 0_1 r_6 3 6
s_cluster 0_2 k_2 2 3
s_cluster 0_2 k_3 3 3
s_cluster 0_2 k_4 4 3
s_cluster 0_2 k_5 5 3
s_cluster 0_2 k_6 6 3
s_cluster 0_2 r_2 3 2
s_cluster 0_2 r_3 3 3
s_cluster 0_2 r_4 3 4
s_cluster 0_2 r_5 3 5
s_cluster 0_2 r_6 3 6
s_cluster 0_3 k_2 2 3
s_cluster 0_3 k_3 3 3
s_cluster 0_3 k_4 4 3
s_cluster 0_3 k_5 5 3
s_cluster 0_3 k_6 6 3
s_cluster 0_3 r_2 3 2
s_cluster 0_3 r_3 3 3
s_cluster 0_3 r_4 3 4
s_cluster 0_3 r_5 3 5
s_cluster 0_3 r_6 3 6


In [134]:
##############################   单用户 单文件  的停留点   ####################
## 输入 st_pa   所有停留点  三层list  [user][file][st_p]
## 输出 st_poi_f  五元组list   [id, lat, lng,time, category]

#### 从总停留点 st_pa 提取用户文件的停留点
##   用户： 000；  文件： 48， 80， 138， 147， 168 (j = 168-1)    
##   

kd = 2        # k 匿名度

o_l = 1     #  最低高度
o_r = 3     #  最高高度 ###########################
cluster = "mb_kmeans"    # 聚类器
ex_times = "0_1"      #第几次实验
at = "k_2"      # 参数设置
i = 0


wz = [47,79,137,146,167]
for aa in wz:
    j = aa
    #j = 167   # 第48个文件  48   138   168

    filename1 = ".\\os_file\\res\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\st_poi_f_"+str(i)+"_"+str(j)+".pkl"
    filename2 = ".\\os_file\\res\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\si_l_f_"+str(i)+"_"+str(j)+".pkl"
    filename3 = ".\\os_file\\res\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\si_h_f_"+str(i)+"_"+str(j)+".pkl"
    filename4 = ".\\os_file\\res\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\k_as_"+str(i)+"_"+str(j)+".pkl" 
    filename5 = ".\\os_file\\res\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\rec_tra_"+str(i)+"_"+str(j)+".pkl"
    filename6 = ".\\os_file\\res\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\rec_cos_"+str(i)+"_"+str(j)+".pkl"
    filename7 = ".\\os_file\\res\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\tra_leaf_"+str(i)+"_"+str(j)+".pkl"

    st_pa_f = []
#st_p = []    #  停留点的list 

    m = st_pa

    if len(m[i]) == 0:            #  如果  user i 没有文件 
        print("该用户没有轨迹文件")
        sys.exit()
    else:       
        if len(m[i][j]) == 0:       #  文件内没有stay point，
            print("该文件内没有停留点")
            sys.exit()
        else:      #  文件内有 stay point               #  user_file_st-p
            for k in range(0, len(m[i][j])):
                            # 对用户 i 文件 j, 停留点 k, st_p[i][j][k]     停留点
                a = []
                a = m[i][j][k]
                st_pa_f.append(a)
            
    b = 0                            
    b = len(st_pa_f)
    print("用户： ",i," 在文件： ",j," 中共有停留点有： ", b,"  个")

#################  匹配 st_pa_f ----> st_poi_f ############### 
############   把 文件停留点， 经 st_poi_dic 匹配, 得 POI+ category： st_poi_f

    st_poi_f = []
    for l in range(0, len(st_pa_f)):
#for i in range(0, 20):
    
        st_p_m = match_1(st_pa_f[l], st_poi_dic)
        st_poi_f.append(st_p_m)

    write_q(st_poi_f, filename1)


##############################  semantic importance  ##################
## 输入 st_poi_f  五元组-两维list   [id, lat, lng,time, category] ;    st_poi_pro_p, st_poi_pro
## 输出 si_l_f    六元组-两维list   [id, lat, lng,time, category,level]
                  # 输出文件 si_l_f_0_47.pkl   

    f = st_poi_f
    pro_p = st_poi_pro_p
    pro = st_poi_pro

    si_l_f = []
    for k in range(len(f)):
    
        ca = f[k][4]
    
        flag = 0
    
        for item1 in pro:
        
            if flag == 1:
                break
            else:
            
                for item2 in pro[item1]:
                    if flag == 1:
                        break
                    else:
                        if ca == item2:         # 判断语义类型匹配
                            flag = 1
                            a = 0
                            a = pro[item1][item2]
                            b = 0
                            b = pro_p[item1][item2]
                        
                            if a == 0:
                                print("st_poi_pro 没有该类型的停留点")
                            
                            c=0
                            c = b * math.log(1/a)
                        
                            d = [f[k][0],f[k][1],f[k][2],f[k][3],f[k][4],c]
                            si_l_f.append(d)
                    
    write_q(si_l_f, filename2)

###########################  si_h_f 语义树高度   #######################
## 输入 si_l_f， o_l, o_r   
## 输出 si_h_f     六元组-两维list   [id, lat, lng,time, category,height]
                  # 输出文件 si_h_f_0_47.pkl 

    al = []
    for k in range(len(si_l_f)):
        al.append(si_l_f[k][5])
    w = max(al)

    si_h_f = []
    for k in range(len(si_l_f)):
        a = si_l_f[k][5]
        b = a/w
        c = 1-b
    
        l = b * o_l + c *o_r
        h = round(l)
    
        d = []
        d = [si_l_f[k][0],si_l_f[k][1],si_l_f[k][2],si_l_f[k][3],si_l_f[k][4],h]
        si_h_f.append(d)

    write_q(si_h_f, filename3)

#####################  轨迹重构 rec_tra  rec_tra_i_j.pkl ################
# 输入  si_h_f 文件内停留点的语义高度； st 语义树；
# 重构轨迹 rec_tra  

    si = si_h_f    # 文件中停留点list 6元组
    st_l = st.leaves()   # st 叶子结点list

    rec_tra = []
    k_as = []
    rec_cos = []
    tra_leaf = []
    for k in range(len(si)):
    
        h = si[k][5]      # 语义匿名高度
        n_id = match_2(si[k], st_l)     # 文件停留点与叶子结点匹配，   返回匹配结点  id
        t_l = st.get_node(n_id)
        tra_leaf.append(t_l)
    #print(st.level(n_id)+1, h)
        m_id = n_id
        if st.level(m_id) < h:
            m_id = "poi"
        else:
            for l in range(h):
            #print(l)
                m_id = st.parent(m_id).identifier   # 返回 中间结点 id
    
    #print(m_id)
        m_l = st.leaves(m_id)    # 返回 中间结点 对应的 叶子结点  匿名集  [node, node, node]
    ##############  删除 空结点  #############
        ai = 0
        while ai < len(m_l):     #  删除空叶子结点
        
            if m_l[ai].data == None:
                del m_l[ai]
            else:
                ai += 1
   
    ########################################
        for l in range(len(m_l)):         ## staying 特征概率化
            abc = m_l[l].data["staying"]
            bcd = proba(abc)
            m_l[l].data["staying"] = bcd
    ########################### 选点  ##############################   
    #  对轨迹第一个点，生成点只要在1500范围内即可
    #  对轨迹中连续的相同点，条件同上
    #  对轨迹中非同一点，角要小于 53 度
        k_as_m = gen_k_as(si[k], m_l, kd)       # 生成K-匿名集，  [node,node,node]
        
        #print("si_h_f:", si_h_f)
        t2 = [si[k][1],si[k][2]]  
        #print("文件：",j,"当前是第",k,"个点")
        if k == 0:
            flag = 0
            while flag == 0:
                idx = np.random.randint(len(k_as_m))      # k范围内，随机生成索引
                r2 = [k_as_m[idx].data["location"]["lat"],k_as_m[idx].data["location"]["lng"]]
                
                dis = Dis([t2, r2])
                #print("轨迹第一个停留点， 生成点与原始点距离为：", dis)
                
                if dis < 1500:
                    flag = 1
        else:
            t1 = [si[k-1][1],si[k-1][2]]
            t1_t2 = Dis([t1,t2])
            
            if t1_t2 == 0:      #如果是同一个停留点，则只要距离不超过  1500 米就行
                #print("该轨迹点与前一个点相同")
                flag = 0
                while flag == 0:
                    
                    idx = np.random.randint(len(k_as_m))      # k范围内，随机生成索引
                    r2 = [k_as_m[idx].data["location"]["lat"],k_as_m[idx].data["location"]["lng"]]
                    
                    t2_r2 = Dis([t2,r2])
                    #print("该轨迹点与生成点距离为： ", t2_r2)
                    if t2_r2 < 1500:
                        flag = 1
            else:
                
                #print("该轨迹点既不是第一个，与前一个点也不相同")
                tcos = 0
                while tcos < 0.59:
                    idx = np.random.randint(len(k_as_m))      # k范围内，随机生成索引
                    r2 = [k_as_m[idx].data["location"]["lat"],k_as_m[idx].data["location"]["lng"]]
            
                    t1_r2 = Dis([t1,r2])
                    t2_r2 = Dis([t2,r2])
                    
                    if t1_r2 == 0:
                        continue
                    
                    dis = [t1_t2, t1_r2, t2_r2]
                    tcos = tra_cos(dis)
                    #print("当前生成点与原始轨迹两个点的偏角余弦为：",tcos)
                rec_cos.append(tcos)

        rec_tra.append(k_as_m[idx])
        k_as.append(k_as_m)
        
    write_q(tra_leaf, filename7)
    write_q(rec_tra, filename5)
    write_q(k_as,filename4)
    write_q(rec_cos,filename6)
    print("用户在文件: ",j,"中共有重构位置：",len(rec_tra),"个")

用户：  0  在文件：  47  中共有停留点有：  14   个
用户在文件:  47 中共有重构位置： 14 个
用户：  0  在文件：  79  中共有停留点有：  10   个
用户在文件:  79 中共有重构位置： 10 个
用户：  0  在文件：  137  中共有停留点有：  11   个
用户在文件:  137 中共有重构位置： 11 个
用户：  0  在文件：  146  中共有停留点有：  9   个
用户在文件:  146 中共有重构位置： 9 个
用户：  0  在文件：  167  中共有停留点有：  13   个
用户在文件:  167 中共有重构位置： 13 个


In [55]:
len(rec_cos)

9

In [21]:
a=[]
b= st.get_node("bar")
a.append(b)
a

[Node(tag=Bar, identifier=bar, data=None)]

In [130]:
##############################   单文件的停留点   ####################
## 输入 st_pa   所有停留点  三层list  [user][file][st_p]
## 输出 st_poi_f  五元组list   [id, lat, lng,time, category]

#### 从总停留点 st_pa 提取用户文件的停留点
##   用户： 000；  文件： 48， 80， 138， 147， 168 (j = 168-1)    
##   

kd = 2        # k 匿名度

o_l = 1     #  最低高度
o_r = 3     #  最高高度 ###########################

i = 0

#a = [{1:47},{1:79},{1:137},{1:146},{1:167}]
#for t in range(len(a)):
j = 146
    #j = 167   # 第48个文件  48   138   168

filename1 = ".\\os_file\\res\\0\\"+"r_3"+ "\\st_poi_f_"+str(i)+"_"+str(j)+".pkl"
filename2 = ".\\os_file\\res\\0\\"+"r_3"+"\\si_l_f_"+str(i)+"_"+str(j)+".pkl"
filename3 = ".\\os_file\\res\\0\\"+"r_3"+"\\si_h_f_"+str(i)+"_"+str(j)+".pkl"
filename4 = ".\\os_file\\res\\0\\"+"r_3"+"\\k_as_"+str(i)+"_"+str(j)+".pkl" 
filename5 = ".\\os_file\\res\\0\\"+"r_3"+"\\rec_tra_"+str(i)+"_"+str(j)+".pkl" 

st_pa_f = []
#st_p = []    #  停留点的list 

m = st_pa

if len(m[i]) == 0:            #  如果  user i 没有文件 
    print("该用户没有轨迹文件")
    sys.exit()
else:       
    if len(m[i][j]) == 0:       #  文件内没有stay point，
        print("该文件内没有停留点")
        sys.exit()
    else:      #  文件内有 stay point               #  user_file_st-p
        for k in range(0, len(m[i][j])):
                            # 对用户 i 文件 j, 停留点 k, st_p[i][j][k]     停留点
            a = []
            a = m[i][j][k]
            st_pa_f.append(a)
            
b = 0                            
b = len(st_pa_f)
print("用户： ",i," 在文件： ",j," 中共有停留点有： ", b,"  个")

#################  匹配 st_pa_f ----> st_poi_f ############### 
############   把 文件停留点， 经 st_poi_dic 匹配, 得 POI+ category： st_poi_f

st_poi_f = []
for l in range(0, len(st_pa_f)):
#for i in range(0, 20):
    
    st_p_m = match_1(st_pa_f[l], st_poi_dic)
    st_poi_f.append(st_p_m)

write_q(st_poi_f, filename1)


##############################  semantic importance  ##################
## 输入 st_poi_f  五元组-两维list   [id, lat, lng,time, category] ;    st_poi_pro_p, st_poi_pro
## 输出 si_l_f    六元组-两维list   [id, lat, lng,time, category,level]
                  # 输出文件 si_l_f_0_47.pkl   

f = st_poi_f
pro_p = st_poi_pro_p
pro = st_poi_pro

si_l_f = []
for k in range(len(f)):
    
    ca = f[k][4]
    
    flag = 0
    
    for item1 in pro:
        
        if flag == 1:
            break
        else:
            
            for item2 in pro[item1]:
                if flag == 1:
                    break
                else:
                    if ca == item2:         # 判断语义类型匹配
                        flag = 1
                        a = 0
                        a = pro[item1][item2]
                        b = 0
                        b = pro_p[item1][item2]
                        
                        if a == 0:
                            print("st_poi_pro 没有该类型的停留点")
                            
                        c=0
                        c = b * math.log(1/a)
                        
                        d = [f[k][0],f[k][1],f[k][2],f[k][3],f[k][4],c]
                        si_l_f.append(d)
                    
write_q(si_l_f, filename2)

###########################  si_h_f 语义树高度   #######################
## 输入 si_l_f， o_l, o_r   
## 输出 si_h_f     六元组-两维list   [id, lat, lng,time, category,height]
                  # 输出文件 si_h_f_0_47.pkl 

al = []
for k in range(len(si_l_f)):
    al.append(si_l_f[k][5])
w = max(al)

si_h_f = []
for k in range(len(si_l_f)):
    a = si_l_f[k][5]
    b = a/w
    c = 1-b
    
    l = b * o_l + c *o_r
    h = round(l)
    
    d = []
    d = [si_l_f[k][0],si_l_f[k][1],si_l_f[k][2],si_l_f[k][3],si_l_f[k][4],h]
    si_h_f.append(d)

write_q(si_h_f, filename3)

#####################  轨迹重构 rec_tra  rec_tra_i_j.pkl ################
# 输入  si_h_f 文件内停留点的语义高度； st 语义树；
# 重构轨迹 rec_tra  

si = si_h_f    # 文件中停留点list 6元组
st_l = st.leaves()   # st 叶子结点list

rec_tra = []
k_as = []
for k in range(len(si)):
    
    h = si[k][5]      # 语义匿名高度
    n_id = match_2(si[k], st_l)     # 文件停留点与叶子结点匹配，   返回匹配结点  id
    #print(st.level(n_id)+1, h)
    m_id = n_id
    if st.level(m_id) < h:
        m_id = "poi"
    else:
        for l in range(h):
            #print(l)
            m_id = st.parent(m_id).identifier   # 返回 中间结点 id
    
    #print(m_id)
    m_l = st.leaves(m_id)    # 返回 中间结点 对应的 叶子结点  匿名集  [node, node, node]
    ##############  删除 空结点  #############
    ai = 0
    while ai < len(m_l):     #  删除空叶子结点
        
        if m_l[ai].data == None:
            del m_l[ai]
        else:
            ai += 1
   
    ########################################
    for l in range(len(m_l)):         ## staying 特征概率化
        abc = m_l[l].data["staying"]
        bcd = proba(abc)
        m_l[l].data["staying"] = bcd
    ############################################    
    #print(len(m_l))
    k_as_m = gen_k_as(si[k], m_l, kd)       # 生成K-匿名集，  [node,node,node]
    
    idx = np.random.randint(kd)      # k范围内，随机生成索引
    
    rec_tra.append(k_as_m[idx])
    k_as.append(k_as_m)
    
write_q(rec_tra, filename5)
write_q(k_as,filename4)
print("用户在文件: ",j,"中共有重构位置：",len(rec_tra),"个")

用户：  0  在文件：  146  中共有停留点有：  9   个
用户在文件:  146 中共有重构位置： 9 个


In [108]:
len(rec_tra)

14

In [81]:
filename = ".\\os_file\\res\\0\\"+"r_6"+"\\k_as_"+str(i)+"_"+str(j)+".pkl"

In [83]:
a= read_q(filename)
len(a[0])

3

In [91]:
filename = ".\\1.pkl"
a = 5
write_q(a, filename)


In [92]:
b=read_q(filename)
b

5

In [18]:
filename1 = ".\\os_file\\st_poi_f_0_47.pkl"
filename2 = ".\\os_file\\si_l_f_0_47.pkl"
filename3 = ".\\os_file\\si_h_f_0_47.pkl"
#filename4 = ".\\os_file\\si_l_f_0_47.pkl"

In [23]:
st_h_f = read_q(filename3)
st_h_f

[['无尽', 40.005973, 116.32533400000001, 65.81666663987562, 'scenic', 2],
 ['绮春园-舞台', 40.010327000000004, 116.314555, 19.833333265269175, 'scenic', 2],
 ['北京大学附属小学-体育馆', 40.000085, 116.32512, 105.74999999371357, 'bbulid', 3],
 ['格林豪泰酒店(首都机场航站楼店)', 40.082075, 116.59334, 10.333333286689594, 'ehotel', 2],
 ['绮春园-舞台', 40.010327000000004, 116.314555, 16.333333337679505, 'scenic', 2],
 ['绮春园-舞台', 40.010327000000004, 116.314555, 22.5, 'scenic', 2],
 ['万春园丙区', 40.007911, 116.31916399999999, 10.3333334333729, 'scenic', 2],
 ['生冬室', 40.009062, 116.315025, 342.0833333779592, 'scenic', 2],
 ['生冬室', 40.009062, 116.315025, 136.0833333141636, 'scenic', 2],
 ['生冬室', 40.009062, 116.315025, 407.08333325455897, 'scenic', 2],
 ['颐和园-治镜阁遗址',
  39.997733000000004,
  116.270547,
  12.166666692355648,
  'scenic',
  2],
 ['北京郊野园中园', 39.996209, 116.26688600000001, 13.75000013387762, 'farm', 1],
 ['宏都商场', 39.908799, 116.40641000000001, 38.08333324617706, 'smarket', 1],
 ['天安门广场', 39.909652, 116.404177, 11.66666659

In [13]:
#############################  计算个人的 st_poi_pro_p   #################################3
###########################  个人文件，停留点 list   st_pa_p list  三元组list ################################
# 输入 st_pa   三层list    st_pa[user][file][st_p]
# st_pa_p       停留点list      三元组list       [lat, lng, time]    

i= 1
filename1 = ".\\os_file\\public_data\\"+str(i)+"\\st_pa_p"+str(i)+".pkl"
filename2 = ".\\os_file\\public_data\\"+str(i)+"\\st_poi_p"+str(i)+".pkl"
filename3 = ".\\os_file\\public_data\\"+str(i)+"\\st_poi_dic_p"+str(i)+".pkl"
filename4 = ".\\os_file\\public_data\\"+str(i)+"\\st_poi_tree_p"+str(i)+".pkl"
filename5 = ".\\os_file\\public_data\\"+str(i)+"\\st_poi_pro_p"+str(i)+".pkl"


st_pa_p = []
#st_p = []    #  停留点的list 

m = st_pa

if len(m[i]) == 0:            #  如果  user i 没有文件 
    sys.exit()
else:       
    for j in range(0, len(m[i])):      
                    # 对用户i的文件j, st_p[i][j],     文件
        if len(m[i][j]) == 0:       #  文件内没有stay point，
            continue
        else:      #  文件内有 stay point               #  user_file_st-p
                
            for k in range(0, len(m[i][j])):
                            # 对用户 i 文件 j, 停留点 k, st_p[i][j][k]     停留点
                a = []
                a = m[i][j][k]
                        #print(a)
                st_pa_p.append(a)
                            
b = len(st_pa_p)
print("用户",i,"共有停留点有： ", b,"  个")

write_q(st_pa_p, filename1)

################################   st_poi   停留点 与 POI 匹配  #####################################
#  停留点   st_pa_p        三元组  [lat,lng, time]
#  POI      st_poi_dic     字典列表  [{},{},{}]
#  输出  st_poi_p      停留点对应的POI，及停留时间  [id, lat, lng, time,category]  

   
st_poi_p = []
for j in range(0, len(st_pa_p)):
#for i in range(0, 20):
    
    st_p_m = match_1(st_pa_p[j], st_poi_dic)
    st_poi_p.append(st_p_m)
    
write_q(st_poi_p, filename2)


#############################  st_poi_dic_p   把 st_poi_p 五元组list 转换成 字典列表  ###################### 
#  输入  st_poi_p  根据POI 【id, lat, lng】 对停留时间进行聚集。  
#  输出 st_poi_dic_p  是 字典list        st_poi_dic_p[i][][]    第i个POI，[][]字典的操作
#    POI  {'id': '无尽',
#          'location': {'lat': 40.005973, 'lng': 116.32533400000001},
#          'staying': [68.36666659684852, 113.54999989969656]}

st_poi_dic_p = []
st_poi = st_poi_p

st_poi_dic = []
for k in range(0,len(st_poi)):    #  待匹配的 st_poi list
    num = 0
    for j in range(0, len(st_poi_dic) ):        # 遍历已有列表
        
        if  st_poi[k][0] == st_poi_dic[j]["id"] and st_poi[k][1] == st_poi_dic[j]["location"]["lat"]  and  \
        st_poi[k][2] == st_poi_dic[j]["location"]["lng"]:       # 如果列表中已有该字典条目
            
            st_poi_dic[j]["staying"].append( st_poi[k][3])
            break
        else:
            num += 1
            #continue
    if num == len(st_poi_dic):        # 如果 现有st_poi_dic  list  中没有，则为该POI 添加 一个新的字典条目
        a = {}
        a["id"] = st_poi[k][0]
        a["location"] = {"lat": st_poi[k][1], "lng": st_poi[k][2]}
        a["staying"] = [st_poi[k][3]]
        a["category"] = st_poi[k][4]
        st_poi_dic.append(a)
    #else:
        #continue
st_poi_dic_p = st_poi_dic
write_q(st_poi_dic_p, filename3)

#################################   基于字典 建立 st_poi_tree 分类树   ##########################
"""#  大类 是 字典，  enter["scenic"]； 
#  小类 是 列表，  scenic[{},{},{}];   scenic[2]["staying"]
#  st_poi_dic_p  是  字典list， st_poi = {id: , location: , staying: , category: }"""

st_poi_tree_p = {}
st_poi_dic = st_poi_dic_p

path = ".\\POI_base\\POI\\"                               # 根目录
class1_list = os.listdir(path)                            # 大类列表
createVar = locals()    # 动态生成变量

st_poi_tree = {}

#print("所有大类： ", class1_list)  
for l in range(0, len(class1_list)):
    #print("当前大类： ", class1_list[i])        
    class2_path = path + class1_list[l]                   # 大类路径 ， 每个大类
    class2_list = os.listdir(class2_path)                 # 小类列表
    #print("小类列表： ", class2_list) 
    
    name1 = os.path.splitext(class1_list[l])[0]      # 取出大类名    去掉后缀名
    createVar[name1] = {}   # 创建大类字典
    
    for j in range(0, len(class2_list)):
        class2_file = class2_path +"\\"+ class2_list[j]     # 小类路径 ， 每个小类
        #print("大类： ",class1_list[i],"下的当前小类： ",  class2_list[j])
        #print("小类文件目录： ", class2_file)
        name2 = os.path.splitext(class2_list[j])[0]      # 取出小类名
        createVar[name2] = []      #创建小类  列表
        
        
        for k in range(0, len(st_poi_dic)):         # 对st_poi_dic 进行分类
            
            if st_poi_dic[k]["category"] == name2:
                createVar[name2].append(st_poi_dic[k])
        
        
        
        createVar[name1][name2] = createVar[name2]    # 把小类字典 嵌入 大类字典
        
    st_poi_tree[name1] = createVar[name1]       # 小类遍历完后，把大类字典，嵌入树
        
st_poi_tree_p = st_poi_tree       
write_q(st_poi_tree_p, filename4)


############################  统计语义类型占比  __个人  #################
#  输入 st_poi_tree_p
#  输出 st_poi_pro_p

tree = st_poi_tree_p
pro = copy.deepcopy(tree)

a = 0
for item1 in tree:
    
    for item2 in tree[item1]:
        
        st = tree[item1][item2]
       
        for j in range(len(st)):    # 小类下的每一个 POI，
            
            stay = st[j]["staying"]
            a += len(stay)
            #for j in range(len(stay)):
print("该用户共有：",a,"个访问点")             
        

for item1 in tree:
    
    for item2 in tree[item1]:
        
        st = tree[item1][item2]
        b = 0       
        for j in range(len(st)):    # 小类下的每一个 POI，
            
            stay = st[j]["staying"]
            b += len(stay)
            #for j in range(len(stay)):
        pro[item1][item2] = round(b/a, 5)
st_poi_pro_p = pro

write_q(st_poi_pro_p, filename5)




用户 1 共有停留点有：  136   个
该用户共有： 136 个访问点


In [16]:
i= 0
filename1 = ".\\os_file\\public_data\\"+str(i)+"\\st_pa_p"+str(i)+".pkl"
filename2 = ".\\os_file\\public_data\\"+str(i)+"\\st_poi_p"+str(i)+".pkl"
filename3 = ".\\os_file\\public_data\\"+str(i)+"\\st_poi_dic_p"+str(i)+".pkl"
filename4 = ".\\os_file\\public_data\\"+str(i)+"\\st_poi_tree_p"+str(i)+".pkl"
filename5 = ".\\os_file\\public_data\\"+str(i)+"\\st_poi_pro_p"+str(i)+".pkl"

In [21]:
st_poi_tree_p = read_q(filename4)
st_poi_tree_p

{'enter': {'bar': [],
  'bath': [{'id': '聚美足道',
    'location': {'lat': 40.010055, 'lng': 116.32128600000001},
    'staying': [13.983333270298317],
    'category': 'bath'},
   {'id': '北京锡华商务酒店-康体中心',
    'location': {'lat': 39.995405, 'lng': 116.30925500000001},
    'staying': [17.416666651843116],
    'category': 'bath'}],
  'bbulid': [{'id': '北京大学附属小学-体育馆',
    'location': {'lat': 40.000085, 'lng': 116.32512},
    'staying': [139.11666667554528,
     13.41666662832722,
     336.84999998426065,
     124.30000007618219,
     215.64999994123355,
     212.31666662497446,
     105.74999999371357,
     308.3833333477378,
     181.0833333141636,
     180.0,
     55.23333335760981,
     182.9333334218245,
     185.23333324701525,
     98.53333328501321,
     228.51666661910713,
     30.6999999366235,
     18.04999996558763,
     49.733333266340196,
     229.2333333066199,
     252.41666672634892,
     133.43333328142762,
     303.9500000048429,
     305.19999993033707,
     21.84999998426065

In [22]:
filename = ".\\os_file\\public_data\\st_poi_tree_p0.pkl"
st_poi_tree_p = read_q(filename)
st_poi_tree_p 

{'enter': {'bar': [],
  'bath': [{'id': '聚美足道',
    'location': {'lat': 40.010055, 'lng': 116.32128600000001},
    'staying': [13.983333270298317],
    'category': 'bath'},
   {'id': '北京锡华商务酒店-康体中心',
    'location': {'lat': 39.995405, 'lng': 116.30925500000001},
    'staying': [17.416666651843116],
    'category': 'bath'}],
  'bbulid': [{'id': '北京大学附属小学-体育馆',
    'location': {'lat': 40.000085, 'lng': 116.32512},
    'staying': [139.11666667554528,
     13.41666662832722,
     336.84999998426065,
     124.30000007618219,
     215.64999994123355,
     212.31666662497446,
     105.74999999371357,
     308.3833333477378,
     181.0833333141636,
     180.0,
     55.23333335760981,
     182.9333334218245,
     185.23333324701525,
     98.53333328501321,
     228.51666661910713,
     30.6999999366235,
     18.04999996558763,
     49.733333266340196,
     229.2333333066199,
     252.41666672634892,
     133.43333328142762,
     303.9500000048429,
     305.19999993033707,
     21.84999998426065

In [1]:
a = [1,2,3]
a[1]

2

In [3]:
a = 0
if 1 > 2 and 3 > 0:
    a = 1

a

0

In [4]:
for k in range(3):
     print(k)

0
1
2


In [5]:
a = []
len(a)

0

In [21]:
np.random.randint(2)

1